In [1]:
# === Multi-seed launcher ===
import json, os
cfg = json.load(open("config.json","r",encoding="utf-8"))
seed_list = cfg.get("simulation",{}).get("seed_list",[None])
print("Running seeds:", seed_list)

Running seeds: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [2]:
# Imports
import os, json, time, random, pprint, logging, colorsys, math, nbformat, pygame, pyperclip, csv, hashlib, uuid, subprocess
from pathlib import Path
import os, json, time, random, pprint, logging, colorsys, math, nbformat, pygame, pyperclip
from typing import Tuple, List, Optional, Dict, Any
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, Future
from nbconvert.exporters import PythonExporter
from ollama import chat as ollama_chat
from pydantic import field_validator, model_validator



pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# === Corasat: Global seed control ===
import random, numpy as np
from pathlib import Path
import json

cfg_path = Path("config.json")
CONFIG = json.load(open(cfg_path,"r",encoding="utf-8"))
seed_list = CONFIG.get("simulation",{}).get("seed_list",[None])
print("Seed list:", seed_list)

def set_all_seeds(s):
    if s is None:
        print("No seed set.")
        return
    random.seed(s)
    np.random.seed(s % (2**32 - 1))
    try:
        import torch
        torch.manual_seed(s)
        torch.cuda.manual_seed_all(s)
    except Exception:
        pass
    print(f"Global seed set to {s}")

Seed list: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [4]:
# Logger
# =========================
class TimestampedLogger:
    def __init__(self, log_dir='logs', log_file='simulation.log'):
        date_tag = datetime.now().strftime('%Y-%m-%d')
        log_file = f'simulation_{date_tag}.log'
        log_dir_path = Path(log_dir)
        log_dir_path.mkdir(parents=True, exist_ok=True)
        os.makedirs(log_dir, exist_ok=True)
        self.log_path = log_dir_path / log_file

        root = logging.getLogger()
        for h in list(root.handlers):
            try: h.close()
            except Exception: pass
            root.removeHandler(h)

        try:
            if os.path.exists(self.log_path):
                os.remove(self.log_path)
        except Exception:
            pass

        fh = logging.FileHandler(self.log_path, mode='a', encoding='utf-8', delay=False)
        ch = logging.StreamHandler()

        fmt = logging.Formatter(fmt='%(levelname)s:%(name)s:%(message)s')
        fh.setFormatter(fmt); ch.setFormatter(fmt)
        root.setLevel(logging.INFO)
        root.addHandler(fh); root.addHandler(ch)

        logging.getLogger('httpx').setLevel(logging.INFO)

        self.start_time = time.time()
        self.last_time = self.start_time
        self.log("Logger initialized.")

    def _now(self): return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    def _duration(self):
        current_time = time.time()
        d = current_time - self.last_time
        self.last_time = current_time
        return f"{d:.3f}s"

    def log(self, message):
        logging.info(f"[{self._now()}] (+{self._duration()}) {message}")

LOGGER = TimestampedLogger()


INFO:root:[2025-11-23 17:08:34] (+0.000s) Logger initialized.


In [5]:
# Constants / Config
# =========================
COLORS = ["white", "black"]
FIGURE_TYPES = ["king", "queen", "rook", "bishop", "knight", "pawn"]
DIRECTION_MAP: Dict[str, Tuple[int, int]] = {
    "north": (0, 1), "south": (0, -1), "east": (1, 0), "west": (-1, 0),
    "northeast": (1, 1), "northwest": (-1, 1), "southeast": (1, -1), "southwest": (-1, -1)
}
VALID_DIRECTIONS = set(DIRECTION_MAP.keys())

def direction_from_vector(vector: Tuple[int, int]) -> str:
    for direction, vec in DIRECTION_MAP.items():
        if vec == vector:
            return direction
    return str(vector)

def hsv_to_rgb255(h_deg: float, s: float, v: float) -> Tuple[int,int,int]:
    """h in degrees [0,360), s,v in [0,1] -> (r,g,b) in 0..255"""
    r, g, b = colorsys.hsv_to_rgb(h_deg/360.0, max(0,min(1,s)), max(0,min(1,v)))
    return (int(r*255), int(g*255), int(b*255))

def load_config(config_path: str = "config.json") -> dict:
    LOGGER.log(f"Load Config: {config_path}")
    try:
        with open(config_path, "r", encoding="utf-8") as f:
            cfg = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"Missing config file: {config_path}")

    return cfg

CONFIG = load_config("config.json")


INFO:root:[2025-11-23 17:08:34] (+0.006s) Load Config: config.json


In [6]:
# Notebook Export
# =========================
def _running_in_notebook() -> bool:
    try:
        from IPython import get_ipython
        return get_ipython().__class__.__name__ == "ZMQInteractiveShell"
    except Exception:
        return False

def _find_notebook_path() -> Optional[str]:
    try:
        import ipynbname
        return str(ipynbname.path())
    except Exception:
        pass
    try:
        candidates = [nb_path for nb_path in os.listdir(".") if nb_path.endswith(".ipynb")]
        if candidates:
            candidates.sort(key=lambda p: os.path.getmtime(p), reverse=True)
            return candidates[0]
    except Exception:
        pass
    return None

def _export_notebook_to_py(nb_path: str, out_py: str = "run_simulation.py") -> None:
    try:
        nb = nbformat.read(nb_path, as_version=4)
        body, _ = PythonExporter().from_notebook_node(nb)
        with open(out_py, "w", encoding="utf-8") as f:
            f.write(body)
        LOGGER.log(f"Exported notebook '{nb_path}' -> '{out_py}'")
    except Exception as e:
        LOGGER.log(f"ERROR: Notebook export failed: {e}")

try:
    if CONFIG["simulation"].get("create_py_export", True):
        if _running_in_notebook():
            nbp = _find_notebook_path()
            if nbp:
                _export_notebook_to_py(nbp, out_py="run_simulation.py")
except Exception:
    pass

# Results Export Helpers
# =========================
RESULTS_FIELDS = [
    "run_id",
    "timestamp",
    "commit_sha",
    "config_hash",
    "model",
    "seed",
    "rounds",
    "broadcasts",
    "coverage",
    "correct_edges",
    "false_edges",
    "total_gt_edges",
    "mission_score",
    "norm_score",
    "runtime_s",
    "logfile"
    ]
try:
    _RESULTS_BASE = Path(__file__).resolve().parent
except NameError:
    _RESULTS_BASE = (Path("Code") / "corasat") if (Path("Code") / "corasat").exists() else Path.cwd()
RESULTS_PATH = _RESULTS_BASE / "results.csv"

def _safe_commit_sha() -> Optional[str]:
    try:
        return subprocess.check_output(["git", "rev-parse", "HEAD"], stderr=subprocess.DEVNULL).decode("utf-8").strip()
    except Exception:
        return None

def _config_hash_from_dict(cfg: Dict[str, Any]) -> Optional[str]:
    try:
        cfg_txt = json.dumps(cfg, sort_keys=True)
        return hashlib.sha1(cfg_txt.encode("utf-8")).hexdigest()[:10]
    except Exception:
        return None

def _compute_coverage_ratio(sim) -> Optional[float]:
    try:
        total = sim.grid_size[0] * sim.grid_size[1]
        if not total:
            return None
        visited = set()
        for drone in getattr(sim, "drones", []):
            for pos in getattr(drone, "mission_report", []):
                if isinstance(pos, (list, tuple)) and len(pos) == 2:
                    visited.add(tuple(pos))
        return round(len(visited) / total, 4)
    except Exception:
        return None

def _build_run_id(seed: Optional[Any]) -> str:
    base = datetime.now().strftime("%Y%m%d-%H%M%S")
    seed_part = str(seed) if seed is not None else "noseed"
    return f"{base}-{seed_part}-{uuid.uuid4().hex[:6]}"

def _append_results_rows(rows: List[Dict[str, Any]]) -> None:
    if not rows:
        return
    RESULTS_PATH.parent.mkdir(parents=True, exist_ok=True)
    file_exists = RESULTS_PATH.exists()
    with open(RESULTS_PATH, "a", newline="", encoding="utf-8") as fh:
        writer = csv.DictWriter(fh, fieldnames=RESULTS_FIELDS)
        if not file_exists:
            writer.writeheader()
        for row in rows:
            writer.writerow(row)

def persist_run_results(run_exports: List[Dict[str, Any]]) -> None:
    if not run_exports:
        return
    commit_sha = _safe_commit_sha()
    logfile = getattr(LOGGER, "log_path", None)
    rows: List[Dict[str, Any]] = []
    for entry in run_exports:
        sim = entry.get("sim")
        config = entry.get("config")
        seed = entry.get("seed")
        runtime_s = entry.get("runtime_s")
        timestamp = entry.get("timestamp") or datetime.now().isoformat()
        coverage = _compute_coverage_ratio(sim) if sim else None
        row = {
            "run_id": _build_run_id(seed),
            "timestamp": timestamp,
            "commit_sha": commit_sha,
            "config_hash": _config_hash_from_dict(config) if config else None,
            "model": getattr(sim, "model", None) if sim else None,
            "seed": seed,
            "rounds": getattr(sim, "round", None) if sim else None,
            "coverage": coverage,
            "correct_edges": getattr(sim, "correct_edge_counter", None) if sim else None,
            "false_edges": getattr(sim, "false_edge_counter", None) if sim else None,
            "total_gt_edges": len(getattr(sim, "gt_edges", []) or []),
            "broadcasts": getattr(sim, "broadcast_count", None) if sim else None,
            "mission_score": getattr(sim, "score", None) if sim else None,
            "norm_score": round(getattr(sim, "score", 0) / len(getattr(sim, "gt_edges", []) or []), 5) if sim and getattr(sim, "gt_edges", None) else None,
            "runtime_s": round(runtime_s, 2) if isinstance(runtime_s, (int, float)) else None,
            "logfile": logfile,
        }
        rows.append(row)
    _append_results_rows(rows)
    LOGGER.log(f"results.csv updated with {len(rows)} run(s).")


In [7]:
# Figure Images
# =========================
FIGURE_IMAGES: Dict[Tuple[str,str], "pygame.Surface"] = {}

def load_figure_images() -> dict:
    images = {}
    base_path = CONFIG["gui"].get("figure_image_dir", "figures")

    def try_load(path):
        return pygame.image.load(path) if os.path.exists(path) else None

    for color in COLORS:
        for figure_type in FIGURE_TYPES:
            candidates = [
                f"{color}{figure_type}.png",
                f"{color.capitalize()}{figure_type}.png",
                f"{color}{figure_type.capitalize()}.png",
                f"{color.capitalize()}{figure_type.capitalize()}.png"
            ]
            img = None
            for name in candidates:
                p = os.path.join(base_path, name)
                img = try_load(p)
                if img:
                    break
            if img:
                images[(color, figure_type)] = img
            else:
                LOGGER.log(f"ERROR: Image not found for {color} {figure_type} in {base_path}")
    return images


In [8]:
# Domain: Figures/Tiles
# =========================
class _Figure:
    def __init__(self, position: Tuple[int, int], color: str, figure_type: str):
        self.position = position
        self.color = color
        self.figure_type = figure_type
        self.defended_by = 0
        self.attacked_by = 0
        self.target_positions: List[Tuple[int, int]] = []

    def calculate_figure_targets(self, board: List[List['_Tile']]):
        self.target_positions = []
        W, H = CONFIG["board"]["width"], CONFIG["board"]["height"]

        def on_board(x, y): return 0 <= x < W and 0 <= y < H

        if self.figure_type in ("queen", "rook", "bishop"):
            if self.figure_type == "rook":
                directions = [(1,0),(-1,0),(0,1),(0,-1)]
            elif self.figure_type == "bishop":
                directions = [(1,1),(-1,-1),(1,-1),(-1,1)]
            else:
                directions = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(1,-1),(-1,1)]
            for dx, dy in directions:
                x, y = self.position
                while True:
                    x += dx; y += dy
                    if not on_board(x, y): break
                    self.target_positions.append((x, y))
                    if board[x][y].figure is not None:
                        break

        elif self.figure_type == "knight":
            for dx, dy in [(2,1),(2,-1),(-2,1),(-2,-1),(1,2),(1,-2),(-1,2),(-1,-2)]:
                x = self.position[0] + dx
                y = self.position[1] + dy
                if on_board(x, y): self.target_positions.append((x, y))

        elif self.figure_type == "king":
            for dx, dy in [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(1,-1),(-1,1)]:
                x = self.position[0] + dx
                y = self.position[1] + dy
                if on_board(x, y): self.target_positions.append((x, y))

        elif self.figure_type == "pawn":
            diagonals = [(1,1),(-1,1)] if self.color == "white" else [(1,-1),(-1,-1)]
            for dx, dy in diagonals:
                x = self.position[0] + dx
                y = self.position[1] + dy
                if on_board(x, y): self.target_positions.append((x, y))

class _Tile:
    def __init__(self, x: int, y: int):
        self.x = x; self.y = y
        self.targeted_by = {"white": 0, "black": 0}
        self.figure = None
        self.drones = []

    def set_figure(self, figure): self.figure = figure
    def add_drone(self, drone):
        if drone not in self.drones: self.drones.append(drone)
    def remove_drone(self, drone):
        if drone in self.drones: self.drones.remove(drone)
    def reset_targeted_by_amounts(self): self.targeted_by = {"white": 0, "black": 0}
    def add_targeted_by_amount(self, color: str, amount: int = 1): self.targeted_by[color] += amount


In [9]:
# Chess helpers
# =========================
def cartesian_to_chess(pos: Tuple[int,int]) -> str:
    x, y = pos
    col = chr(ord('a') + x)
    row = str(y + 1)
    return f"{col}{row}"

def chess_to_cartesian(s: str) -> Tuple[int,int]:
    s = s.strip().lower()
    if len(s) < 2: raise ValueError(f"Invalid chess pos: {s}")
    col, row = s[0], s[1:]
    x = ord(col) - ord('a')
    y = int(row) - 1
    return (x, y)

def format_edge(source_type: str, source_color: str, target_color: str, edge: Tuple[Tuple[int,int], Tuple[int,int]]) -> str:
    src, dst = edge
    piece_symbol = {
        "king": "K", "queen": "Q", "rook": "R",
        "bishop": "B", "knight": "N", "pawn": ""
    }.get(source_type, "?")
    capture_symbol = "x" if source_color != target_color else "-"
    return f"{piece_symbol}{cartesian_to_chess(src)}{capture_symbol}{cartesian_to_chess(dst)}"

def on_board(x, y):
    return 0 <= x < CONFIG["board"]["width"] and 0 <= y < CONFIG["board"]["height"]


In [10]:
# Drone
# =========================
class _Drone:
    def __init__(self, id: int, position: Tuple[int, int], model: str, rules: str, sim, color: str = "white"):
        self.id = id
        self.position = position
        self.color = color
        self.model = model
        self.sim = sim
        self.rules = rules.replace("DRONE_ID", str(self.id))\
                          .replace("NUMBER_OF_DRONES", str(CONFIG["simulation"]["num_drones"]))\
                          .replace("NUMBER_OF_ROUNDS", str(CONFIG["simulation"]["max_rounds"]))
        self.memory = ""
        self.rx_buffer = ""
        self.mission_report: List = [self.position]
        self.assigned_sector = self._default_sector_assignment()
        self.rendezvous_directive: Optional[dict] = None
        self.mission_plan: List[dict] = self._build_initial_mission_plan()
        self.current_leg_index: int = 0
        self.info_exchange_rounds: Dict[int, Optional[int]] = {
            drone_id: None
            for drone_id in range(1, CONFIG["simulation"]["num_drones"] + 1)
            if drone_id != self.id
        }
        self._last_decision_support: Dict[str, object] = {}
        self.local_board = {}
        for bx in range(CONFIG["board"]["width"]):
            for by in range(CONFIG["board"]["height"]):
                self.local_board[cartesian_to_chess((bx, by))] = {"color": "unknown", "type": "unknown"}
        # color may be "unknown", "n/a", "white", "black"
        # type may be "unknown", "n/a", "any figure", "king", "queen", "rook", "bishop", "knight", "pawn", "a possible target"
        # possible combinations: (unknown,unknown), (n/a,n/a), (any figure,white/black), (king/queen/rook/bishop/knight/pawn,white/black), (a possible target,unknown)
        self.identified_edges = []
        self.update_board_report()

    def _default_sector_assignment(self) -> Dict[str, Any]:
        board_w = CONFIG["board"]["width"]
        board_h = CONFIG["board"]["height"]
        return {
            "upper_left": cartesian_to_chess((0, board_h - 1)),
            "lower_right": cartesian_to_chess((board_w - 1, 0)),
            "label": "Full board",
        }

    def _normalize_sector_point(self, value: Any) -> Optional[str]:
        if value is None:
            return None
        board_w = CONFIG["board"]["width"]
        board_h = CONFIG["board"]["height"]
        if isinstance(value, str):
            value = value.strip()
            if not value:
                return None
            try:
                x, y = chess_to_cartesian(value)
            except Exception:
                return None
        elif isinstance(value, (list, tuple)) and len(value) == 2:
            try:
                x = int(value[0])
                y = int(value[1])
            except (TypeError, ValueError):
                return None
        else:
            return None
        x = max(0, min(board_w - 1, x))
        y = max(0, min(board_h - 1, y))
        if not on_board(x, y):
            return None
        return cartesian_to_chess((x, y))

    def _normalize_sector_assignment(self, sector: Any) -> Dict[str, Any]:
        normalized = dict(self._default_sector_assignment())
        if not sector:
            return normalized
        if isinstance(sector, str):
            label = sector.strip()
            if label:
                normalized["label"] = label
            return normalized
        if isinstance(sector, dict):
            label = sector.get("label") or sector.get("name")
            if isinstance(label, str) and label.strip():
                normalized["label"] = label.strip()
            ul = sector.get("upper_left") or sector.get("ul")
            lr = sector.get("lower_right") or sector.get("lr")
            ul_norm = self._normalize_sector_point(ul)
            lr_norm = self._normalize_sector_point(lr)
            if ul_norm:
                normalized["upper_left"] = ul_norm
            if lr_norm:
                normalized["lower_right"] = lr_norm
        return normalized

    def _sector_bounds(self, sector: Optional[Dict[str, Any]] = None) -> Optional[Tuple[int, int, int, int]]:
        descriptor = sector or self.assigned_sector
        if not descriptor:
            return None

        def _to_cart(coord: Any) -> Optional[Tuple[int, int]]:
            if coord is None:
                return None
            if isinstance(coord, str):
                try:
                    return chess_to_cartesian(coord)
                except Exception:
                    return None
            if isinstance(coord, (list, tuple)) and len(coord) == 2:
                try:
                    return (int(coord[0]), int(coord[1]))
                except (TypeError, ValueError):
                    return None
            return None

        ul = _to_cart(descriptor.get("upper_left") or descriptor.get("ul"))
        lr = _to_cart(descriptor.get("lower_right") or descriptor.get("lr"))
        if not ul or not lr:
            return None
        board_w = CONFIG["board"]["width"]
        board_h = CONFIG["board"]["height"]
        min_x = max(0, min(ul[0], lr[0]))
        max_x = min(board_w - 1, max(ul[0], lr[0]))
        min_y = max(0, min(ul[1], lr[1]))
        max_y = min(board_h - 1, max(ul[1], lr[1]))
        if min_x > max_x or min_y > max_y:
            return None
        return (min_x, max_x, min_y, max_y)

    def _sector_changed(self, old: Optional[Dict[str, Any]], new: Dict[str, Any]) -> bool:
        if old is None:
            return True
        return any(old.get(key) != new.get(key) for key in ("upper_left", "lower_right"))

    def _sector_to_string(self, sector: Optional[Dict[str, Any]]) -> str:
        if not sector:
            return "Full board"
        ul = sector.get("upper_left") or sector.get("ul") or "?"
        lr = sector.get("lower_right") or sector.get("lr") or "?"
        label = sector.get("label")
        coords_text = f"{ul} -> {lr}"
        return f"{label} ({coords_text})" if label else coords_text

    def sector_summary(self) -> str:
        return self._sector_to_string(self.assigned_sector)

    def _set_sector_assignment(self, sector: Any) -> None:
        normalized = self._normalize_sector_assignment(sector)
        changed = self._sector_changed(getattr(self, "assigned_sector", None), normalized)
        self.assigned_sector = normalized
        if changed:
            self.mission_plan = self._build_mission_plan_for_sector(normalized)
            LOGGER.log(f"Drone {self.id} adopted sector {self.sector_summary()}")
        self.current_leg_index = 0 if changed else min(self.current_leg_index, max(0, len(self.mission_plan) - 1))
        self._sanitize_leg_index()

    def _apply_rendezvous_to_plan(self, plan: List[dict]) -> List[dict]:
        waypoint = self.rendezvous_directive
        if not waypoint:
            return plan
        filtered: List[dict] = [wp for wp in plan if wp.get("notes") != "Rendezvous directive."]
        rv_turn = waypoint.get("turn", CONFIG["simulation"].get("max_rounds", 1))
        rv_copy = dict(waypoint)
        inserted = False
        for idx, wp in enumerate(filtered):
            if wp.get("turn", 0) > rv_turn:
                filtered.insert(idx, rv_copy)
                inserted = True
                break
        if not inserted:
            filtered.append(rv_copy)
        return filtered

    def _sanitize_leg_index(self) -> None:
        if not self.mission_plan:
            self.current_leg_index = 0
            return
        self.current_leg_index = max(0, min(self.current_leg_index, len(self.mission_plan) - 1))

    def _current_mission_leg(self) -> Optional[dict]:
        if not self.mission_plan:
            return None
        self._sanitize_leg_index()
        return self.mission_plan[self.current_leg_index]

    def _advance_leg_progress(self) -> None:
        while True:
            leg = self._current_mission_leg()
            if not leg:
                return
            tolerance = max(1, int(leg.get("tolerance_steps", 1)))
            target_vec = leg.get("leg_end") or leg.get("target_cartesian")
            if not target_vec:
                return
            target = tuple(target_vec)
            if chebyshev_distance(self.position, target) <= tolerance:
                if self.current_leg_index < len(self.mission_plan) - 1:
                    self.current_leg_index += 1
                    continue
                self.current_leg_index = len(self.mission_plan) - 1
            return

    def _leg_segment_bounds(self, leg: dict) -> Optional[Tuple[Tuple[int, int], Tuple[int, int]]]:
        try:
            start_vec = leg.get("leg_start") or leg.get("start_cartesian")
            end_vec = leg.get("leg_end") or leg.get("target_cartesian")
            if start_vec is None or end_vec is None:
                return None
            start = (int(start_vec[0]), int(start_vec[1]))
            end = (int(end_vec[0]), int(end_vec[1]))
            return start, end
        except (TypeError, ValueError):
            return None

    @staticmethod
    def _distance_to_point(a: Tuple[int, int], b: Tuple[int, int]) -> int:
        return chebyshev_distance(a, b)

    def _project_to_leg(self, pos: Tuple[int, int], leg: dict) -> Optional[Tuple[int, int]]:
        bounds = self._leg_segment_bounds(leg)
        if not bounds:
            target_vec = leg.get("target_cartesian")
            if target_vec:
                return tuple(target_vec)
            return None
        (sx, sy), (ex, ey) = bounds
        if sx == ex:
            min_y = min(sy, ey)
            max_y = max(sy, ey)
            x = sx
            y = min(max(pos[1], min_y), max_y)
            return (x, y)
        if sy == ey:
            min_x = min(sx, ex)
            max_x = max(sx, ex)
            x = min(max(pos[0], min_x), max_x)
            y = sy
            return (x, y)
        return tuple(bounds[1])

    def _distance_to_leg(self, pos: Tuple[int, int], leg: Optional[dict]) -> Optional[int]:
        if leg is None:
            return None
        projection = self._project_to_leg(pos, leg)
        if projection is None:
            target_vec = leg.get("target_cartesian")
            if target_vec:
                return self._distance_to_point(pos, tuple(target_vec))
            return None
        return self._distance_to_point(pos, projection)

    def _distance_to_leg_end(self, pos: Tuple[int, int], leg: Optional[dict]) -> Optional[int]:
        if leg is None:
            return None
        target_vec = leg.get("leg_end") or leg.get("target_cartesian")
        if not target_vec:
            return None
        return self._distance_to_point(pos, tuple(target_vec))

    def _build_initial_mission_plan(self) -> List[dict]:
        return self._build_mission_plan_for_sector(self.assigned_sector)

    def _build_mission_plan_for_sector(self, sector: Optional[Dict[str, Any]]) -> List[dict]:
        bounds = self._sector_bounds(sector)
        if not bounds:
            sector = self._default_sector_assignment()
            bounds = self._sector_bounds(sector)
            self.assigned_sector = sector
        min_x, max_x, min_y, max_y = bounds
        board_w = CONFIG["board"]["width"]
        board_h = CONFIG["board"]["height"]

        def _col_label(x: int) -> str:
            return chr(ord('a') + max(0, min(board_w - 1, x)))

        def _row_label(y: int) -> int:
            return y + 1

        def _clamp_to_sector(pos: Tuple[int, int]) -> Tuple[int, int]:
            return (
                max(min_x, min(max_x, pos[0])),
                max(min_y, min(max_y, pos[1]))
            )

        plan: List[dict] = []
        leg_id = 1
        current_turn = 1
        entry_point = _clamp_to_sector(self.position)
        current_position = entry_point
        entry_source = tuple(self.position)

        def _append_leg(start: Tuple[int, int], end: Tuple[int, int], orientation: str, note: str) -> None:
            nonlocal leg_id, current_turn, plan, current_position
            if start == end and orientation in {"horizontal", "vertical"}:
                orientation = "point"
            if start == end:
                length = 0
            elif orientation == "vertical":
                length = abs(end[1] - start[1])
            elif orientation == "horizontal":
                length = abs(end[0] - start[0])
            else:
                length = max(abs(end[0] - start[0]), abs(end[1] - start[1]))
            tolerance = 0
            entry = {
                "leg_id": leg_id,
                "turn": current_turn,
                "start": cartesian_to_chess(start),
                "start_cartesian": [start[0], start[1]],
                "target": cartesian_to_chess(end),
                "target_cartesian": [end[0], end[1]],
                "leg_start": [start[0], start[1]],
                "leg_end": [end[0], end[1]],
                "orientation": orientation,
                "length": length,
                "tolerance_steps": tolerance,
                "notes": note,
            }
            plan.append(entry)
            leg_id += 1
            current_turn += max(1, length)
            current_position = end

        def _transit_to(target: Tuple[int, int], reason: str) -> None:
            nonlocal current_position
            if current_position == target:
                return
            if current_position[0] != target[0]:
                intermediate = (target[0], current_position[1])
                direction = "east" if target[0] > current_position[0] else "west"
                _append_leg(
                    current_position,
                    intermediate,
                    "horizontal",
                    f"{reason} (horizontal {direction})"
                )
                current_position = intermediate
            if current_position[1] != target[1]:
                intermediate = (current_position[0], target[1])
                direction = "north" if target[1] > current_position[1] else "south"
                _append_leg(
                    current_position,
                    intermediate,
                    "vertical",
                    f"{reason} (vertical {direction})"
                )
                current_position = intermediate

        if entry_point != entry_source:
            source_label = cartesian_to_chess(entry_source)
            entry_label = cartesian_to_chess(entry_point)
            _append_leg(
                entry_point,
                entry_point,
                "point",
                f"Sector entry anchor at {entry_label} (from {source_label})"
            )

        def _edge_distance(point: Tuple[int, int], start: Tuple[int, int], end: Tuple[int, int]) -> int:
            if start[0] == end[0]:
                return abs(point[0] - start[0])
            return abs(point[1] - start[1])

        def _edge_label(start: Tuple[int, int], end: Tuple[int, int]) -> str:
            return f"{cartesian_to_chess(start)} -> {cartesian_to_chess(end)}"

        upper_left = (min_x, max_y)
        upper_right = (max_x, max_y)
        lower_right = (max_x, min_y)
        lower_left = (min_x, min_y)
        perimeter_edges = [
            {"name": "north", "start": upper_left, "end": upper_right, "orientation": "horizontal"},
            {"name": "east", "start": upper_right, "end": lower_right, "orientation": "vertical"},
            {"name": "south", "start": lower_right, "end": lower_left, "orientation": "horizontal"},
            {"name": "west", "start": lower_left, "end": upper_left, "orientation": "vertical"},
        ]

        valid_edge_indices = [
            idx for idx, edge in enumerate(perimeter_edges)
            if edge["start"] != edge["end"]
        ]

        start_idx = 0
        if valid_edge_indices:
            start_idx = min(
                valid_edge_indices,
                key=lambda idx: _edge_distance(entry_point, perimeter_edges[idx]["start"], perimeter_edges[idx]["end"])
            )

        ordered_edges = perimeter_edges[start_idx:] + perimeter_edges[:start_idx]

        for edge in ordered_edges:
            if edge["start"] == edge["end"]:
                continue
            start = edge["start"]
            end = edge["end"]
            orientation = edge["orientation"]
            start_label = cartesian_to_chess(start)
            edge_label = _edge_label(start, end)
            _transit_to(start, f"Stage to {edge['name']} edge start {start_label}")
            _append_leg(
                start,
                end,
                orientation,
                f"Counter-clockwise perimeter sweep along {edge['name']} edge ({edge_label})"
            )

        if not plan:
            origin = (min_x, min_y)
            _append_leg(
                origin,
                origin,
                "point",
                "Hold position within single-tile sector."
            )

        plan = self._apply_rendezvous_to_plan(plan)
        self.current_leg_index = 0
        self.mission_plan = plan
        self._sanitize_leg_index()
        self._advance_leg_progress()
        return plan

    def _next_mission_waypoint(self) -> Optional[dict]:
        return self._current_mission_leg()

    def _decision_support_snapshot(self) -> Dict[str, object]:
        snapshot = compute_decision_support(self)
        self._last_decision_support = snapshot
        return snapshot


    def _format_decision_support_lines(self, snapshot: Dict[str, object]) -> Tuple[List[str], List[str]]:
        lines: List[str] = []
        ledger_lines: List[str] = []

        waypoint = snapshot.get("next_waypoint")
        if waypoint:
            tolerance = waypoint.get("tolerance_steps", 0)
            if waypoint.get("leg_start") and waypoint.get("leg_end"):
                leg_id = waypoint.get("leg_id", "?")
                start = waypoint.get("start")
                if not start:
                    start_vec = waypoint.get("leg_start")
                    start = cartesian_to_chess(tuple(start_vec)) if start_vec else "?"
                target = waypoint.get("target")
                if not target:
                    end_vec = waypoint.get("leg_end")
                    target = cartesian_to_chess(tuple(end_vec)) if end_vec else "?"
                lines.append(
                    f"Plan focus: leg {leg_id} {start} -> {target} (turn {waypoint.get('turn','?')}, tol +/-{tolerance})"
                )
            else:
                lines.append(
                    f"Plan focus: {waypoint.get('target','?')} by turn {waypoint.get('turn','?')} (tol +/-{tolerance})"
                )

        if getattr(self, 'assigned_sector', None):
            lines.append(f"Sector directive: {self.sector_summary()}")

        suggestion = snapshot.get("coordination_suggestion")
        if suggestion:
            lines.append("Suggested coordination broadcast JSON:")
            plan_json = json.dumps(suggestion.get("plan", {}), indent=2)
            for line in plan_json.splitlines():
                lines.append(f"  {line}")

        score_entries = snapshot.get("scores", [])
        sorted_scores = sorted(
            score_entries,
            key=lambda entry: entry.get("score", float('-inf')),
            reverse=True,
        )

        def _qualitative_label(rank: int, score: float) -> str:
            if rank == 0:
                return "Best choice"
            if score >= 5:
                return "Excellent choice"
            if score >= 2:
                return "Good choice"
            if score >= 0:
                return "Okay choice"
            if score > -2:
                return "Risky choice"
            return "Bad choice"

        def _format_score(value: float) -> str:
            formatted = f"{value:.2f}"
            if "." in formatted:
                formatted = formatted.rstrip("0").rstrip(".")
            return formatted

        for idx, entry in enumerate(sorted_scores):
            components = entry.get("components", {})
            if components:
                sorted_components = sorted(
                    components.items(),
                    key=lambda item: (-abs(item[1]), item[0])
                )
                component_text = ", ".join(
                    f"{name} ({value:+.2f})" for name, value in sorted_components[:3]
                )
            else:
                component_text = "n/a"
            notes = entry.get("notes") or []
            note_text = " | notes: " + "; ".join(notes) if notes else ""
            action = (entry.get("action") or "-").strip()
            label = (entry.get("label") or "").strip()
            if action == "move" and label:
                action_text = f"{action} {label}"
            elif action in {"broadcast", "wait"}:
                action_text = action
            elif label and label != "-":
                action_text = f"{action} {label}"
            else:
                action_text = action
            qualitative = _qualitative_label(idx, entry.get("score", 0.0))
            score_text = _format_score(entry.get("score", 0.0))
            lines.append(
                f"{action_text} -> {qualitative} ({score_text}) | factors: {component_text}{note_text}"
            )

        for ledger in snapshot.get("intel_ledger", []):
            last_round = ledger.get("last_round")
            age = ledger.get("age", 0)
            if last_round is None:
                ledger_lines.append(
                    f"Drone {ledger['drone']}: never shared (age {age} rounds)"
                )
            else:
                ledger_lines.append(
                    f"Drone {ledger['drone']}: last shared round {last_round} (age {age})"
                )

        return lines, ledger_lines
    def _identify_edges(self):
        # Identify edges according to current local board knowledge
        directions = []
        for bx in range(CONFIG["board"]["width"]):
            for by in range(CONFIG["board"]["height"]):
                # Identify figure
                figure_type = self.local_board[cartesian_to_chess((bx,by))]["type"]
                figure_color = self.local_board[cartesian_to_chess((bx,by))]["color"]
                if not figure_type in FIGURE_TYPES: continue # Only consider known figures
                if figure_type in ("queen", "rook", "bishop"): # Sliding pieces
                    is_slider = True
                    if figure_type == "rook":
                        directions = [(1,0),(-1,0),(0,1),(0,-1)]
                    elif figure_type == "bishop":
                        directions = [(1,1),(-1,-1),(1,-1),(-1,1)]
                    else:
                        directions = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(1,-1),(-1,1)]
                else:
                    is_slider = False
                    if figure_type == "knight":
                        directions = [(2,1),(2,-1),(-2,1),(-2,-1),(1,2),(1,-2),(-1,2),(-1,-2)]
                    elif figure_type == "king":
                        directions = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(1,-1),(-1,1)]
                    elif figure_type == "pawn":
                        directions = [(1,1),(-1,1)] if figure_color == "white" else [(1,-1),(-1,-1)]

                # Virtually move figure
                for dx, dy in directions:
                    tx, ty = bx, by
                    while True:
                        tx += dx
                        ty += dy
                        if not on_board(tx, ty): break
                        if self.local_board[cartesian_to_chess((tx,ty))]["type"] == "unknown": # a possible target (prioritize investigation)
                            self.local_board[cartesian_to_chess((tx,ty))]["type"] = "a possible target"
                        if self.local_board[cartesian_to_chess((tx,ty))]["type"] == "a possible target": # Moving beyond this is pointless for sliders
                            break
                        if self.local_board[cartesian_to_chess((tx,ty))]["type"] in FIGURE_TYPES \
                            or self.local_board[cartesian_to_chess((tx,ty))]["type"] == "any figure": # Found an edge
                            edge = format_edge(
                                figure_type,
                                self.local_board[cartesian_to_chess((bx,by))]["color"],
                                self.local_board[cartesian_to_chess((tx,ty))]["color"],
                                ((bx,by),(tx,ty)))
                            if edge not in self.identified_edges and (bx,by)!=(tx,ty):
                                self.identified_edges.append(edge)
                                if edge not in self.sim.reported_edges:
                                    if edge in self.sim.gt_edges:
                                        correct_marker = "- CORRECT: "
                                    else:
                                        correct_marker = "- FALSE: "
                                    edge_identified_message = f"{correct_marker}{edge}"
                                    LOGGER.log(edge_identified_message)
                                    if self.sim.gui:
                                        self.sim.gui.post_info(edge_identified_message)
                            break
                        if not is_slider: break

    def update_board_report(self):
        # Current position (see color and type)
        sx, sy = self.position
        tile = self.sim.board[sx][sy]
        if tile.figure:
            self.local_board[cartesian_to_chess((sx,sy))] = {"color": tile.figure.color, "type": tile.figure.figure_type}
        else:
            self.local_board[cartesian_to_chess((sx,sy))] = {"color": "n/a", "type": "n/a"}

        # Neighboring tiles (see color, no type)
        for dx in [-1,0,1]:
            for dy in [-1,0,1]:
                if dx==0 and dy==0: continue
                nx, ny = sx+dx, sy+dy
                if not on_board(nx, ny): continue
                neighbor_tile = self.sim.board[nx][ny]
                if neighbor_tile.figure:
                    if self.local_board[cartesian_to_chess((nx,ny))]["type"] == "unknown":
                        self.local_board[cartesian_to_chess((nx,ny))] = {"color": neighbor_tile.figure.color, "type": "any figure"}
                else:
                    self.local_board[cartesian_to_chess((nx,ny))] = {"color": "n/a", "type": "n/a"}

        # Identify edges according to board knowledge
        self._identify_edges()

        # Report to simulation
        self.sim.report_edges(self.identified_edges)

    def _apply_plan_directive(self, payload: Dict[str, object]) -> None:
        if not isinstance(payload, dict):
            return

        plan = payload.get("plan") if isinstance(payload.get("plan"), dict) else None
        if plan is None:
            return

        assignments = plan.get("assignments")
        if isinstance(assignments, list):
            for assignment in assignments:
                if not isinstance(assignment, dict):
                    continue
                drone_ref = assignment.get("drone")
                if drone_ref is None:
                    continue
                if str(drone_ref).strip() == str(self.id):
                    sector = assignment.get("sector")
                    if sector is not None:
                        self._set_sector_assignment(sector)
                    break

        rendezvous = plan.get("rendezvous")
        if isinstance(rendezvous, dict):
            tile = rendezvous.get("tile")
            turn = rendezvous.get("turn")
            if tile:
                try:
                    cart = chess_to_cartesian(str(tile))
                    if not on_board(*cart):
                        LOGGER.log(f"Drone {self.id} received out-of-bounds rendezvous '{tile}'")
                        return
                    max_rounds = max(1, CONFIG["simulation"].get("max_rounds", 1))
                    requested_turn = int(turn) if turn is not None else max_rounds - 1
                    rv_turn = max(1, max_rounds - 1)
                    if rv_turn != requested_turn:
                        LOGGER.log(f"Drone {self.id} set rendezvous turn to {rv_turn} to reserve the final round for intel broadcast")
                    waypoint = {
                        "turn": rv_turn,
                        "target": cartesian_to_chess(cart),
                        "target_cartesian": [cart[0], cart[1]],
                        "distance_steps": chebyshev_distance(self.position, cart),
                        "tolerance_steps": 0,
                        "notes": "Rendezvous directive."
                    }
                    self.rendezvous_directive = waypoint

                    self.mission_plan = self._apply_rendezvous_to_plan(self.mission_plan)
                    LOGGER.log(f"Drone {self.id} adopted rendezvous {waypoint['target']} by turn {rv_turn}")
                except Exception as exc:
                    LOGGER.log(f"Drone {self.id} failed to apply rendezvous directive: {exc}")
        self._sanitize_leg_index()

    def _provide_intelligence_to(self, target_drone: '_Drone'):
        # Share local board knowledge
        for pos, info in self.local_board.items():
            tgt_info = target_drone.local_board.get(pos, {"color": "unknown", "type": "unknown"})
            if tgt_info["color"] == "unknown" and info["color"] != "unknown":
                target_drone.local_board[pos]["color"] = info["color"]
            if tgt_info["type"] == "unknown" and info["type"] != "unknown":
                target_drone.local_board[pos]["type"] = info["type"]

        # Share identified edges
        for edge in self.identified_edges:
            if edge not in target_drone.identified_edges:
                target_drone.identified_edges.append(edge)

        current_round = self.sim.round
        self.info_exchange_rounds[target_drone.id] = current_round
        target_drone.info_exchange_rounds[self.id] = current_round
        # Update their board report (in case new edges imply new "a possible target" statuses)
        target_drone.update_board_report()

    def generate_full_model_response(self) -> List[dict]:
        temperature = CONFIG["simulation"].get("temperature", 0.7)
        situation = self._determine_situation_description()

        # Append cues
        promt_requests = CONFIG.get("prompt_requests", {})
        cues = "\n".join([
            promt_requests.get("rationale",""), 
            promt_requests.get("action",""),
            promt_requests.get("action_move",""), 
            promt_requests.get("action_broadcast",""), 
            promt_requests.get("memory_update","")
        ]).strip()
        user_content = situation if not cues else situation + "\n\n" + cues

        messages = [
            {"role": "system", "content": self.rules},
            {"role": "user", "content": user_content}
        ]
        prompt_char_len = len(user_content) + len(self.rules)
        print(f"Context length (chars): {prompt_char_len}")
        return self._generate_single_model_response(
            messages=messages,
            model=self.model,
            temperature=temperature,
            prompt_char_len=prompt_char_len,
        )

    def _legal_movement_steps(self) -> List[dict]:
        sx, sy = self.position
        reachable_tiles = []
        for name, (dx, dy) in DIRECTION_MAP.items():
            nx, ny = sx + dx, sy + dy
            if on_board(nx, ny):
                reachable_tiles.append({
                    "direction": name,
                    "new_position": (nx, ny)
                    })
        return reachable_tiles

    def _move(self, direction: str) -> bool:
        direction = (direction or "").lower()
        if direction in DIRECTION_MAP:
            dx, dy = DIRECTION_MAP[direction]
            nx, ny = self.position[0] + dx, self.position[1] + dy
            if on_board(nx, ny):
                self.sim.board[self.position[0]][self.position[1]].remove_drone(self)
                self.position = (nx, ny)
                self.sim.board[nx][ny].add_drone(self)
                self.mission_report.append(self.position)
                self.update_board_report()
                self._advance_leg_progress()
                LOGGER.log(f"Drone {self.id} moved to {cartesian_to_chess(self.position)}.")
                return True
            else:
                LOGGER.log(f"ERROR: Drone {self.id} attempted to move to {(nx,ny)}.")
        else:
            LOGGER.log(f"ERROR: Drone {self.id} attempted invalid direction '{direction}'.")
        return False

    def _determine_situation_description(self) -> str:
        # Only local, directly observable info + own memory + received broadcasts + gathered intelligence
        same_tile_drones = [
            f"Drone {drone.id}" for drone in self.sim.board[self.position[0]][self.position[1]].drones if drone.id != self.id
        ]

        fig_here = "None"
        if self.sim.board[self.position[0]][self.position[1]].figure:
            fig_here = self.sim.board[self.position[0]][self.position[1]].figure.figure_type

        # neighbor figure colors (visibility model)
        neighbor_figures = ""
        for dx in [-1,0,1]:
            for dy in [-1,0,1]:
                if dx==0 and dy==0:
                    continue
                nx, ny = self.position[0]+dx, self.position[1]+dy
                if on_board(nx, ny):
                    tile = self.sim.board[nx][ny]
                    if tile.figure:
                        neighbor_figures += f"{direction_from_vector((dx,dy))}: {tile.figure.color}, "
        neighbor_figures = neighbor_figures.strip(", ")

        legal_movements = ", ".join([f"{lms['direction']} to {cartesian_to_chess(lms['new_position'])}" for lms in self._legal_movement_steps()])

        collected_figure_information = ""
        for bx in range(CONFIG["board"]["width"]):
            for by in range(CONFIG["board"]["height"]):
                info = self.local_board[cartesian_to_chess((bx,by))]
                if info["type"] in FIGURE_TYPES:
                    collected_figure_information += f"{cartesian_to_chess((bx,by))}: {info['color']} {info['type']}, "
                elif info["type"] == "a possible target":
                    collected_figure_information += f"{cartesian_to_chess((bx,by))}: {info['type']}, "
        collected_figure_information = collected_figure_information.strip(", ")

        s = []
        s.append(f"Current round number: {self.sim.round} of {CONFIG['simulation']['max_rounds']} rounds.")
        s.append(f"Current position: {cartesian_to_chess(self.position)}")
        s.append(f"Legal movements: {legal_movements}")
        # rest of standard context
        s.append(f"Visible drones at position: {', '.join(same_tile_drones) if same_tile_drones else 'None'}")
        s.append(f"Visible figure at position: {fig_here}")
        s.append(f"Visible neighbor figures: {neighbor_figures or 'None'}")
        s.append(f"Memory: {self.memory}")
        s.append(f"Collected figure information: {collected_figure_information}")
        if getattr(self, 'assigned_sector', None):
            s.append(f"Assigned coverage sector: {self.sector_summary()}")
        if getattr(self, 'rendezvous_directive', None):
            rv = self.rendezvous_directive
            s.append(f"Rendezvous directive: {rv['target']} on turn {rv['turn']}")
        s.append(f"Broadcast Rx Buffer: {self.rx_buffer}")
        if self.id == 1 and getattr(self.sim, "round", None) == 1 and getattr(self.sim, "turn", None) == 1:
            total_drones = CONFIG["simulation"].get("num_drones", len(getattr(self.sim, "drones", [])))
            s.append("Special directive: As Drone 1 on the opening turn, broadcast a coverage plan assigning every drone their sector before taking other actions.")
            s.append(f"Ensure the broadcast is valid JSON with plan->assignments entries for all {total_drones} drones (including yourself) and describe each sector clearly.")

        snapshot = self._decision_support_snapshot()
        ds_lines, ledger_lines = self._format_decision_support_lines(snapshot)
        if ds_lines:
            s.append("Decision Support:")
            s.extend([f"  {entry}" for entry in ds_lines])
        if ledger_lines:
            s.append("Intel Share Ledger:")
            s.extend([f"  {entry}" for entry in ledger_lines])
        self.rx_buffer = ""  # drain the inbox each turn
        situation_description = "\n".join(s)
        LOGGER.log(f"Drone {self.id} Situation:\n{situation_description}")
        return situation_description

    def _generate_single_model_response(self, messages: List[dict], model: str, temperature: float, prompt_char_len: Optional[int] = None) -> List[dict]:
        if model == "manual":
            try:
                pyperclip.copy(messages[-1]["content"])
            except Exception:
                pass
            content = input("Paste model result: ")
            messages.append({"role": "assistant", "content": content})
            if prompt_char_len is not None:
                approx_tokens = max(1, math.ceil(prompt_char_len / 4))
                print(f"Context length: ~{approx_tokens} tokens ({prompt_char_len} chars)")
            return messages

        else: # Ollama
            request_started = time.perf_counter()
            response = ollama_chat(model=model, messages=messages, stream=False, format="json",
                               options={"temperature": float(temperature)})
            elapsed = time.perf_counter() - request_started
            content = response["message"]["content"]
            messages.append({"role": "assistant", "content": content})
            prompt_tokens = response.get("prompt_eval_count")
            completion_tokens = response.get("eval_count")
            eval_duration = response.get("eval_duration")
            duration_seconds = None
            if isinstance(eval_duration, (int, float)) and eval_duration > 0:
                duration_seconds = eval_duration / 1_000_000_000
            elif elapsed > 0:
                duration_seconds = elapsed

            if prompt_tokens is not None:
                ctx_msg = f"Context length: {prompt_tokens} tokens"
                if prompt_char_len is not None:
                    ctx_msg += f" ({prompt_char_len} chars)"
                print(ctx_msg)
            elif prompt_char_len is not None:
                approx_tokens = max(1, math.ceil(prompt_char_len / 4))
                print(f"Context length: ~{approx_tokens} tokens ({prompt_char_len} chars)")

            if isinstance(completion_tokens, (int, float)) and completion_tokens > 0 and duration_seconds and duration_seconds > 0:
                tokens_per_second = completion_tokens / duration_seconds
                print(f"Tokens per second: {tokens_per_second:.2f} tok/s ({completion_tokens} completion tokens)")
            return messages


In [11]:
# Decision Support
# =========================

def chebyshev_distance(a: Tuple[int, int], b: Tuple[int, int]) -> int:
    return max(abs(a[0] - b[0]), abs(a[1] - b[1]))

def _count_unknown_neighbors(drone: _Drone, pos: Tuple[int, int]) -> int:
    count = 0
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0:
                continue
            nx, ny = pos[0] + dx, pos[1] + dy
            if not on_board(nx, ny):
                continue
            key = cartesian_to_chess((nx, ny))
            info = drone.local_board.get(key, {"color": "unknown", "type": "unknown"})
            if info["type"] == "unknown" or info["color"] == "unknown" or info["type"] == "a possible target":
                count += 1
    return count


def _distance_to_sector(point: Tuple[int, int], bounds: Tuple[int, int, int, int]) -> Optional[int]:
    if point is None or bounds is None:
        return None
    px, py = point
    min_x, max_x, min_y, max_y = bounds
    dx = 0
    if px < min_x:
        dx = min_x - px
    elif px > max_x:
        dx = px - max_x
    dy = 0
    if py < min_y:
        dy = min_y - py
    elif py > max_y:
        dy = py - max_y
    return max(dx, dy)

def _build_coordination_suggestion(drone: _Drone) -> Dict[str, Any]:
    sim = drone.sim
    board_w = CONFIG["board"]["width"]
    board_h = CONFIG["board"]["height"]
    num_drones = max(1, sim.num_drones)
    block_width = max(1, math.ceil(board_w / num_drones))
    assignments: List[Dict[str, Any]] = []
    for idx in range(num_drones):
        start_col = min(idx * block_width, board_w - 1)
        end_col = min(board_w - 1, start_col + block_width - 1)
        if start_col > end_col:
            start_col = end_col
        upper_left = cartesian_to_chess((start_col, board_h - 1))
        lower_right = cartesian_to_chess((end_col, 0))
        assignments.append({
            "drone": idx + 1,
            "sector": {
                "upper_left": upper_left,
                "lower_right": lower_right,
            }
        })
    rendezvous_turn = max(1, CONFIG["simulation"].get("max_rounds", 1) - 1)
    rendezvous_tile = cartesian_to_chess(sim.drone_base)
    return {
        "plan": {
            "assignments": assignments,
            "rendezvous": {
                "tile": rendezvous_tile,
                "turn": rendezvous_turn,
            }
        }
    }

def compute_decision_support(drone: _Drone) -> Dict[str, object]:
    ds_cfg = CONFIG.get("decision_support", {})
    scoring_cfg = ds_cfg.get("scoring", {})
    board_w = CONFIG["board"]["width"]
    board_h = CONFIG["board"]["height"]

    move_cfg = scoring_cfg.get("move", {})
    broadcast_cfg = scoring_cfg.get("broadcast", {})
    wait_cfg = scoring_cfg.get("wait", {})

    # Move scoring parameters
    waypoint_progress_reward = move_cfg.get("waypoint_progress_reward_per_step", 1.0)  # per step closer
    waypoint_regression_penalty = move_cfg.get("waypoint_regression_penalty_per_step", -1.0) # per step further
    deadline_slack_bonus = move_cfg.get("deadline_slack_bonus", 0.2) # per turn slack
    deadline_slack_penalty = move_cfg.get("deadline_slack_penalty", -1.0) # per turn overdue
    tolerance_bonus = move_cfg.get("tolerance_bonus", 0.5)
    unknown_tile_bonus = move_cfg.get("unknown_tile_bonus", 1.0)
    possible_target_bonus = move_cfg.get("possible_target_bonus", 1.2)
    figure_hint_bonus = move_cfg.get("figure_hint_bonus", 0.6)
    unknown_color_bonus = move_cfg.get("unknown_color_bonus", 0.5)
    unknown_neighbor_bonus = move_cfg.get("unknown_neighbor_bonus_per_tile", 0.2)
    leg_alignment_reward = move_cfg.get("leg_alignment_reward", 0.6)
    leg_alignment_penalty = move_cfg.get("leg_alignment_penalty", -0.6)
    leg_travel_reward = move_cfg.get("leg_travel_reward", 0.9)
    leg_travel_penalty = move_cfg.get("leg_travel_penalty", -0.3)
    leg_sideways_reward = move_cfg.get("leg_sideways_reward", 1.2)
    leg_sideways_probe_bonus = move_cfg.get("leg_sideways_inspection_bonus", leg_sideways_reward)
    leg_along_penalty = move_cfg.get("leg_along_penalty", -0.7)
    leg_start_progress_reward = move_cfg.get("leg_start_progress_reward", 1.2)
    leg_start_regression_penalty = move_cfg.get("leg_start_regression_penalty", -1.0)
    leg_neighbor_scan_bonus = move_cfg.get("leg_neighbor_scan_bonus", 0.4)
    sector_alignment_reward = move_cfg.get("sector_alignment_reward", 0.8)
    sector_inside_bonus = move_cfg.get("sector_inside_bonus", 0.5)
    late_penalty_multiplier = move_cfg.get("late_penalty_multiplier", 2.0)
    border_edge_bonus = move_cfg.get("board_edge_bias_bonus", 0.2)
    border_edge_range = int(move_cfg.get("board_edge_bias_range", 1))
    sector_unknown_probe_bonus = move_cfg.get("sector_unknown_probe_bonus", 0.0)
    sector_unknown_probe_min_slack = float(move_cfg.get("sector_unknown_probe_min_slack", 1))

    # Broadcast scoring parameters
    broadcast_base_penalty = broadcast_cfg.get("base_penalty", -0.5)
    broadcast_recipient_factor = broadcast_cfg.get("recipient_factor", 0.8)
    broadcast_staleness_factor = broadcast_cfg.get("staleness_factor", 0.4)
    coordination_broadcast_bonus = broadcast_cfg.get("first_turn_coordination_bonus", 2.5)
    is_first_coordination_turn = (
        drone.id == 1
        and getattr(drone.sim, "round", None) == 1
        and getattr(drone.sim, "turn", None) == 1
    )

    # Wait scoring parameters
    wait_default_score = wait_cfg.get("default_score", -1.0)
    wait_idle_component = wait_cfg.get("idle_penalty_component", -1.0)
    wait_holding_score = wait_cfg.get("holding_position_score", -0.2)
    wait_holding_component = wait_cfg.get("holding_pattern_component", 0.3)
    wait_over_activity_penalty = wait_cfg.get("over_activity_penalty", 0.0)

    scores: List[Dict[str, object]] = []
    best_move_score = float('-inf')
    current_round = drone.sim.round
    next_wp = drone._next_mission_waypoint()
    target_pos = tuple(next_wp["target_cartesian"]) if next_wp else None
    current_leg = next_wp if next_wp and next_wp.get("leg_start") else None
    current_leg_distance = drone._distance_to_leg(drone.position, current_leg) if current_leg else None
    current_leg_end_distance = drone._distance_to_leg_end(drone.position, current_leg) if current_leg else None
    current_leg_start: Optional[Tuple[int, int]] = None
    current_leg_start_distance: Optional[int] = None
    if current_leg:
        start_vec = current_leg.get("leg_start") or current_leg.get("start_cartesian")
        if isinstance(start_vec, (list, tuple)) and len(start_vec) == 2:
            try:
                current_leg_start = (int(start_vec[0]), int(start_vec[1]))
                current_leg_start_distance = drone._distance_to_point(drone.position, current_leg_start)
            except (TypeError, ValueError):
                current_leg_start = None
                current_leg_start_distance = None

    sector_bounds = drone._sector_bounds() if hasattr(drone, "_sector_bounds") else None
    current_sector_distance = _distance_to_sector(drone.position, sector_bounds) if sector_bounds else None
    sector_unknown_tiles: List[Tuple[int, int]] = []
    nearest_sector_unknown: Optional[Dict[str, object]] = None
    if sector_bounds:
        min_x, max_x, min_y, max_y = sector_bounds
        best_unknown_dist: Optional[int] = None
        best_unknown_pos: Optional[Tuple[int, int]] = None
        for sx in range(min_x, max_x + 1):
            for sy in range(min_y, max_y + 1):
                tile_key = cartesian_to_chess((sx, sy))
                tile_report = drone.local_board.get(tile_key, {})
                if tile_report.get("type") == "unknown":
                    pos = (sx, sy)
                    sector_unknown_tiles.append(pos)
                    dist = chebyshev_distance(drone.position, pos)
                    if best_unknown_dist is None or dist < best_unknown_dist:
                        best_unknown_dist = dist
                        best_unknown_pos = pos
        if best_unknown_pos is not None and best_unknown_dist is not None:
            nearest_sector_unknown = {
                "tile": cartesian_to_chess(best_unknown_pos),
                "distance": best_unknown_dist,
            }

    for step in drone._legal_movement_steps():
        direction = step["direction"]
        new_pos = tuple(step["new_position"])
        tile_key = cartesian_to_chess(new_pos)
        tile_info = drone.local_board.get(tile_key, {"color": "unknown", "type": "unknown"})

        score = 0.0
        score_components: Dict[str, float] = {}
        notes: List[str] = []

        new_start_distance: Optional[int] = None

        if current_leg:
            new_leg_distance = drone._distance_to_leg(new_pos, current_leg)
            if current_leg_start is not None:
                new_start_distance = drone._distance_to_point(new_pos, current_leg_start)
                if current_leg_start_distance is not None and current_leg_start_distance > 0 and new_start_distance is not None:
                    delta_start = current_leg_start_distance - new_start_distance
                    score_components["leg_start_focus"] = round(delta_start, 2)
                    if delta_start > 0:
                        score += leg_start_progress_reward
                        notes.append("closing distance to leg start")
                    elif delta_start < 0:
                        score += leg_start_regression_penalty
                        notes.append("drifting from leg start")
            if current_leg_distance is not None and new_leg_distance is not None:
                delta_leg = current_leg_distance - new_leg_distance
                score_components["leg_alignment"] = round(delta_leg, 2)
                if delta_leg > 0:
                    score += leg_alignment_reward
                    notes.append("aligning with coverage leg")
                elif delta_leg < 0:
                    score += leg_alignment_penalty
                    notes.append("drifting from coverage leg")
                if current_leg_distance and current_leg_distance > 0 and new_leg_distance == 0:
                    notes.append("entered coverage leg corridor")
                orientation = (current_leg.get("orientation") or "").lower()
                dx = new_pos[0] - drone.position[0]
                dy = new_pos[1] - drone.position[1]
                if current_leg_distance and current_leg_distance > 0:
                    if orientation == "vertical":
                        if abs(dx) == 1 and abs(dy) == 0 and new_leg_distance < current_leg_distance:
                            score += leg_sideways_reward
                            notes.append("sidestep toward vertical leg")
                        if abs(dx) == 0 and abs(dy) == 1 and delta_leg <= 0:
                            score += leg_along_penalty
                            notes.append("slide along vertical leg before aligning")
                    elif orientation == "horizontal":
                        if abs(dy) == 1 and abs(dx) == 0 and new_leg_distance < current_leg_distance:
                            score += leg_sideways_reward
                            notes.append("sidestep toward horizontal leg")
                        if abs(dy) == 0 and abs(dx) == 1 and delta_leg <= 0:
                            score += leg_along_penalty
                            notes.append("slide along horizontal leg before aligning")
            if current_leg_distance == 0 and new_leg_distance == 1:
                if abs(new_pos[0] - drone.position[0]) <= 1 and abs(new_pos[1] - drone.position[1]) <= 1:
                    score += leg_sideways_probe_bonus
                    score_components["leg_sideways_probe"] = round(leg_sideways_probe_bonus, 2)
                    notes.append("sideways probe off leg")
            if new_leg_distance == 0 and current_leg_end_distance is not None:
                if new_start_distance is not None and new_start_distance > 0 and current_leg_start_distance is not None and current_leg_start_distance > 0:
                    score += leg_start_regression_penalty
                    notes.append("skipping leg start")
                else:
                    new_end_distance = drone._distance_to_leg_end(new_pos, current_leg)
                    if new_end_distance is not None:
                        delta_end = current_leg_end_distance - new_end_distance
                        if delta_end > 0:
                            score += leg_travel_reward
                            score_components["leg_travel"] = round(delta_end, 2)
                            notes.append("progress along current leg")
                        elif delta_end < 0:
                            score += leg_travel_penalty
                            notes.append("retreat along current leg")

        if target_pos:
            # Evaluate progress toward next waypoint
            current_dist = chebyshev_distance(drone.position, target_pos)
            new_dist = chebyshev_distance(new_pos, target_pos)
            turns_remaining = max(0, next_wp["turn"] - (current_round + 1))
            slack = turns_remaining - new_dist

            delta = current_dist - new_dist
            score_components["plan_progress"] = delta
            if delta > 0:
                score += waypoint_progress_reward
                notes.append("closer to waypoint")
            elif delta < 0:
                regression_penalty = waypoint_regression_penalty
                if slack < 0:
                    regression_penalty *= max(1.0, late_penalty_multiplier)
                score += regression_penalty
                notes.append("further from waypoint")

            score_components["deadline_margin"] = float(slack)
            if slack >= 0:
                score += deadline_slack_bonus
                notes.append("on schedule")
            else:
                slack_penalty = deadline_slack_penalty * max(1.0, late_penalty_multiplier)
                score += slack_penalty
                notes.append("missing deadline")

            # Evaluate rendezvous buffer
            if next_wp.get("notes") == "Rendezvous directive.":
                buffer_turns = max(0, slack)
                score_components["rendezvous_buffer"] = float(buffer_turns)
                suffix = "turn" if buffer_turns == 1 else "turns"
                notes.append(f"{buffer_turns} {suffix} left to reach rendezvous on time")
                if buffer_turns == 0:
                    if slack < 0:
                        notes.append("rendezvous already overdue - move immediately")
                    else:
                        notes.append("critical rendezvous move - no buffer remaining")

            # Evaluate tolerance achievement
            tolerance = next_wp.get("tolerance_steps", 0)
            if new_dist <= tolerance:
                score += tolerance_bonus
                score_components["within_tolerance"] = 1.0

            if (
                sector_unknown_tiles
                and sector_unknown_probe_bonus
                and slack >= sector_unknown_probe_min_slack
            ):
                closest_probe_distance = min(chebyshev_distance(new_pos, pos) for pos in sector_unknown_tiles)
                if slack >= closest_probe_distance:
                    score += sector_unknown_probe_bonus
                    score_components["sector_unknown_probe"] = round(sector_unknown_probe_bonus, 2)
                    notes.append(
                        f"sector probe feasible (nearest unknown {closest_probe_distance} steps away without delaying leg)"
                    )

        # Evaluate tile information gain
        if tile_info["type"] == "unknown":
            score += unknown_tile_bonus
            score_components["discover_type"] = unknown_tile_bonus
            notes.append("unidentified tile")
        elif tile_info["type"] == "a possible target":
            score += possible_target_bonus
            score_components["possible_target"] = possible_target_bonus
            notes.append("possible blocker")
        elif tile_info["type"] == "any figure":
            score += figure_hint_bonus
            score_components["figure_hint"] = figure_hint_bonus
            notes.append("figure nearby")

        # Evaluate color information gain
        if tile_info["color"] == "unknown":
            score += unknown_color_bonus
            score_components["discover_color"] = unknown_color_bonus

        # Evaluate unknown neighbors
        unknown_neighbors = _count_unknown_neighbors(drone, new_pos)
        if unknown_neighbors:
            score += unknown_neighbors * unknown_neighbor_bonus
            score_components["unknown_neighbors"] = float(unknown_neighbors)
            notes.append(f"{unknown_neighbors} unknown neighbors")

        if sector_bounds:
            new_sector_distance = _distance_to_sector(new_pos, sector_bounds)
            if new_sector_distance is not None:
                if (sector_alignment_reward and current_sector_distance is not None
                        and new_sector_distance < current_sector_distance):
                    score += sector_alignment_reward
                    score_components["sector_alignment"] = round(current_sector_distance - new_sector_distance, 2)
                    notes.append("moving toward assigned sector")
                if sector_inside_bonus and new_sector_distance == 0:
                    score += sector_inside_bonus
                    score_components["sector_inside"] = round(sector_inside_bonus, 2)
                    notes.append("within assigned sector")

        if border_edge_bonus and border_edge_range >= 0:
            border_distance = min(
                new_pos[0],
                board_w - 1 - new_pos[0],
                new_pos[1],
                board_h - 1 - new_pos[1],
            )
            if border_distance <= border_edge_range:
                bias_strength = max(1, border_edge_range - border_distance + 1)
                bonus = border_edge_bonus * bias_strength
                score += bonus
                score_components["border_bias"] = round(bonus, 2)
                notes.append("nudged toward board edge")

        best_move_score = max(best_move_score, score)

        scores.append({
            "action": "move",
            "label": direction,
            "score": round(score, 2),
            "components": {k: round(v, 2) for k, v in score_components.items() if abs(v) >= 0.01},
            "notes": notes,
        })

    # Broadcast scoring
    tile = drone.sim.board[drone.position[0]][drone.position[1]]
    recipients = [d for d in tile.drones if d.id != drone.id]
    broadcast_components: Dict[str, float] = {"recipients": float(len(recipients))}
    broadcast_notes: List[str] = []
    broadcast_score = broadcast_base_penalty
    if recipients:
        ages: List[int] = []
        for target in recipients:
            last_round = drone.info_exchange_rounds.get(target.id)
            age = current_round - last_round if last_round is not None else current_round
            if age < 0:
                age = 0
            ages.append(age)
        avg_age = sum(ages) / len(ages) if ages else 0.0
        broadcast_components["avg_staleness"] = round(avg_age, 2)
        broadcast_components["max_staleness"] = float(max(ages) if ages else 0)
        broadcast_score = len(recipients) * broadcast_recipient_factor + avg_age * broadcast_staleness_factor
        if avg_age > 0:
            broadcast_notes.append("recipients have stale intel")
    else:
        broadcast_notes.append("no co-located drones")

    if is_first_coordination_turn and coordination_broadcast_bonus:
        broadcast_score += coordination_broadcast_bonus
        broadcast_components["coordination_bonus"] = float(coordination_broadcast_bonus)
        broadcast_notes.append("first-turn coverage assignment priority")

    scores.append({
        "action": "broadcast",
        "label": "share",
        "score": round(broadcast_score, 2),
        "components": {k: v for k, v in broadcast_components.items()},
        "notes": broadcast_notes,
    })

    # Wait scoring
    wait_score = wait_default_score
    wait_notes: List[str] = ["no progress"]
    wait_components: Dict[str, float] = {"idle_penalty": wait_idle_component}
    if target_pos:
        dist_to_target = chebyshev_distance(drone.position, target_pos)
        if dist_to_target == 0 and next_wp and current_round < next_wp["turn"]:
            wait_score = wait_holding_score
            wait_notes = ["holding position at waypoint"]
            wait_components["holding_pattern"] = wait_holding_component
    scores.append({
        "action": "wait",
        "label": "hold",
        "score": round(wait_score, 2),
        "components": {k: round(v, 2) for k, v in wait_components.items()},
        "notes": wait_notes,
    })

    scores.sort(key=lambda entry: entry["score"], reverse=True)

    intel_ledger: List[Dict[str, object]] = []
    for other_id in sorted(drone.info_exchange_rounds.keys()):
        last_round = drone.info_exchange_rounds[other_id]
        age = current_round - last_round if last_round is not None else current_round
        if age < 0:
            age = 0
        intel_ledger.append({
            "drone": other_id,
            "last_round": last_round,
            "age": age,
        })

    coordination_suggestion: Optional[Dict[str, Any]] = None
    if is_first_coordination_turn:
        try:
            coordination_suggestion = _build_coordination_suggestion(drone)
        except Exception:
            coordination_suggestion = None

    return {
        "scores": scores,
        "next_waypoint": next_wp,
        "intel_ledger": intel_ledger,
        "coordination_suggestion": coordination_suggestion,
        "sector_unknown_target": nearest_sector_unknown,
    }


In [12]:
# GUI
# =========================
class _SimulationGUI:
    def __init__(self, sim):
        self.sim = sim
        self.grid_size = (CONFIG["board"]["width"], CONFIG["board"]["height"])
        self.info_lines: List[str] = []
        self.info_max_lines = 10000
        self.info_scroll = 0

        pygame.init()

        gui = CONFIG["gui"]
        self.sidebar_width = gui.get("sidebar_width", 480)
        flags = pygame.HWSURFACE | pygame.DOUBLEBUF

        cell = gui["cell_size"]; m = gui["margin"]
        board_w = self.grid_size[0]*(cell+m) + m
        board_h = self.grid_size[1]*(cell+m) + m
        total_w = board_w + self.sidebar_width
        total_h = board_h

        self.screen = pygame.display.set_mode((total_w, total_h), flags)
        pygame.display.set_caption(self.sim.progress_caption(round_num=1, turn_num=1))
        self.clock = pygame.time.Clock()

        # Fonts
        try:
            self._font = pygame.font.SysFont(None, 18)
            self._font_small = pygame.font.SysFont(None, 16)
        except Exception:
            pygame.font.init()
            self._font = pygame.font.Font(None, 18)
            self._font_small = pygame.font.Font(None, 16)

        # Images
        global FIGURE_IMAGES
        if not FIGURE_IMAGES:
            FIGURE_IMAGES = load_figure_images()
        for k, surf in list(FIGURE_IMAGES.items()):
            try:
                FIGURE_IMAGES[k] = surf.convert_alpha()
            except Exception:
                pass
        self._image_cache = {}

    # ---- utilities
    def _get_scaled(self, img):
        key = (id(img), CONFIG["gui"]["cell_size"])
        if key not in self._image_cache:
            self._image_cache[key] = pygame.transform.scale(
                img, (CONFIG["gui"]["cell_size"], CONFIG["gui"]["cell_size"])
            )
        return self._image_cache[key]

    def post_info(self, text: str):
        for line in text.splitlines():
            self.info_lines.append(line)
        if len(self.info_lines) > self.info_max_lines:
            self.info_lines = self.info_lines[-self.info_max_lines:]
        self.info_scroll = 0

    def _tile_center_px(self, x: int, y: int) -> Tuple[int,int]:
        gui = CONFIG["gui"]
        cell = gui["cell_size"]; m = gui["margin"]
        _, gh = self.grid_size
        y_flip = gh - 1 - y
        cx = x*(cell+m) + m + cell//2
        cy = y_flip*(cell+m) + m + cell//2
        return (cx, cy)

    def save_screenshot(self, path: Optional[str] = None):
        out_dir = "screenshots"
        os.makedirs(out_dir, exist_ok=True)
        fname = path or os.path.join(out_dir, f"last_run.png")
        try:
            pygame.image.save(self.screen, fname)
            LOGGER.log(f"Saved GUI screenshot: {fname}")
        except Exception as e:
            LOGGER.log(f"Failed to save screenshot: {e}")

    # ---- sidebar
    def _draw_score_panel(self, x0: int, y0: int, w: int) -> int:
        s = self.screen
        pad = 12
        y = y0

        s.blit(self._font.render("Score", True, (220, 220, 220)), (x0 + pad, y)); y += 24
        items = [
            ("True edges", len(self.sim.gt_edges)),
            ("Reported edges", len(self.sim.reported_edges)),
            ("- Correct", self.sim.correct_edge_counter),
            ("- False", self.sim.false_edge_counter),
            ("Score", self.sim.score)
        ]
        for lbl, val in items:
            s.blit(self._font_small.render(f"{lbl}: {val}", True, (200, 200, 200)), (x0 + pad, y))
            y += 18

        y += 6
        s.blit(self._font.render("Plans", True, (220, 220, 220)), (x0 + pad, y)); y += 20
        current_round = self.sim.round
        for drone in self.sim.drones:
            drone_color = getattr(drone, "render_color", (200, 200, 50))
            sw = pygame.Rect(x0 + pad, y + 2, 10, 10)
            pygame.draw.rect(self.screen, drone_color, sw)
            label = f"  Drone {drone.id}"
            if getattr(drone, "color", None):
                label += f" [{drone.color}]"
            s.blit(self._font_small.render(label, True, (200, 200, 200)), (x0 + pad + 14, y))
            y += 16
            if getattr(drone, "assigned_sector", None):
                try:
                    sector_text = drone.sector_summary()
                except Exception:
                    sector_text = "n/a"
                s.blit(self._font_small.render(f"    Sector: {sector_text}", True, (150, 180, 200)), (x0 + pad + 14, y))
                y += 16

            next_wp = drone._next_mission_waypoint() if hasattr(drone, "_next_mission_waypoint") else None
            plan_entries = []
            for idx, wp in enumerate(getattr(drone, "mission_plan", [])):
                if idx < getattr(drone, "current_leg_index", 0):
                    continue
                plan_entries.append(wp)
            if not plan_entries:
                s.blit(self._font_small.render("    (no upcoming waypoints)", True, (140, 140, 140)), (x0 + pad + 14, y))
                y += 16
            else:
                for wp in plan_entries[:4]:
                    marker = ">>" if wp is next_wp else "  "
                    target = wp.get("target") or "?"
                    start = wp.get("start")
                    if not start:
                        start_vec = wp.get("leg_start")
                        start = cartesian_to_chess(tuple(start_vec)) if start_vec else "?"
                    if not target:
                        end_vec = wp.get("leg_end")
                        target = cartesian_to_chess(tuple(end_vec)) if end_vec else "?"
                    turn = wp.get("turn", "?")
                    tolerance = wp.get("tolerance_steps")
                    note = wp.get("notes")
                    tol_suffix = ""
                    if isinstance(tolerance, (int, float)) and tolerance > 0:
                        tol_suffix = f" (+/-{tolerance})"
                    leg_label = f"L{wp.get('leg_id','?')}"
                    orientation = wp.get("orientation")
                    orient_token = f", {orientation[0].upper()}" if isinstance(orientation, str) and orientation else ""
                    text = f"{marker} {leg_label} {start}->{target} (T{turn}{orient_token}){tol_suffix}"
                    s.blit(self._font_small.render(f"    {text}", True, (180, 180, 180)), (x0 + pad + 14, y))
                    y += 16
                    if note:
                        s.blit(self._font_small.render(f"      {note}", True, (120, 120, 120)), (x0 + pad + 14, y))
                        y += 16
            y += 6

        y += 8
        s.blit(self._font_small.render("Latest at bottom", True, (160, 160, 160)), (x0 + w - 150, y0 + 2))
        pygame.draw.line(s, (60,60,60), (x0, y), (x0 + w, y), 1)
        return y + 10

    def _draw_sidebar(self):
        gui = CONFIG["gui"]
        s = self.screen

        cell = gui["cell_size"]; m = gui["margin"]
        board_w = self.grid_size[0]*(cell+m) + m
        x0 = board_w
        y0 = 0
        w = self.sidebar_width
        h = s.get_height()

        pygame.draw.rect(s, (25,25,25), pygame.Rect(x0, y0, w, h))
        pygame.draw.line(s, (60,60,60), (x0, 0), (x0, h), 1)

        y_log_top = self._draw_score_panel(x0, y0 + 8, w)

        pad = 12
        tx = x0 + pad
        ty = y_log_top + 8
        tw = w - 2 * pad
        th = h - ty - pad

        pygame.draw.rect(s, (32,32,32), pygame.Rect(tx-4, ty-4, tw+8, th+8))

        line_h = self._font_small.get_height() + 4
        max_lines_fit = max(1, th // line_h)
        start_idx = max(0, len(self.info_lines) - max_lines_fit - self.info_scroll)
        end_idx = len(self.info_lines) - self.info_scroll

        y = ty
        for line in self.info_lines[start_idx:end_idx]:
            if y > ty + th - line_h:
                break
            s.blit(self._font_small.render(line, True, (230,230,230)), (tx, y))
            y += line_h

    # ---- board
    def draw_field(self):
        gui = CONFIG["gui"]
        cell = gui["cell_size"]
        m = gui["margin"]
        gw, gh = self.grid_size

        # clear
        self.screen.fill(gui["background_color"])

        # 1) draw tiles and figures first (paths must overlap tiles)
        for x in range(gw):
            for y in range(gh):
                y_flip = gh - 1 - y
                rect = pygame.Rect(x * (cell + m) + m, y_flip * (cell + m) + m, cell, cell)
                pygame.draw.rect(self.screen, gui["grid_color"], rect)

                tile = self.sim.board[x][y]

                # figure image or fallback
                if tile.figure:
                    img = FIGURE_IMAGES.get((tile.figure.color, tile.figure.figure_type))
                    if img:
                        key = (id(img), cell)
                        if key not in self._image_cache:
                            self._image_cache[key] = pygame.transform.scale(img, (cell, cell))
                        self.screen.blit(self._image_cache[key], rect.topleft)
                    else:
                        pygame.draw.circle(self.screen, (200, 200, 200), rect.center, cell // 3)

                # attacked/defended overlay
                if tile.figure:
                    fig = tile.figure
                    overlay = f"D{fig.defended_by} A{fig.attacked_by}"
                    surf = self._font_small.render(overlay, True, gui["text_color"])
                    pad = 3
                    tx = rect.left + pad
                    ty = rect.bottom - surf.get_height() - pad
                    pygame.draw.rect(self.screen, (40, 40, 40),
                                    pygame.Rect(tx - 2, ty - 1, surf.get_width() + 4, surf.get_height() + 2))
                    self.screen.blit(surf, (tx, ty))

        # 2) draw drone paths
        def _path_offset_vec(drone) -> Tuple[int, int]:
            n = max(1, len(self.sim.drones))
            k = (drone.id - 1) % n
            ang = math.radians(30.0 + 360.0 * k / n)  # diagonal base, evenly spaced by ID
            r = max(2, int(cell * 0.12))              # small pixel offset within tile
            return (int(r * math.cos(ang)), int(r * math.sin(ang)))

        for drone in self.sim.drones:
            if len(drone.mission_report) >= 2:
                base_pts = [ self._tile_center_px(x, y) for (x, y) in drone.mission_report ]
                ox, oy = _path_offset_vec(drone)
                pts = [(x + ox, y + oy) for (x, y) in base_pts]
                color = getattr(drone, "render_color", gui["drone_color"])
                try:
                    pygame.draw.lines(self.screen, color, False, pts, 2)
                except Exception:
                    pass
                # start marker
                sx, sy = pts[0]
                pygame.draw.circle(self.screen, color, (sx, sy), 3)

        # 3) draw drones on top of paths
        for x in range(gw):
            for y in range(gh):
                y_flip = gh - 1 - y
                rect = pygame.Rect(x * (cell + m) + m, y_flip * (cell + m) + m, cell, cell)
                tile = self.sim.board[x][y]

                total = len(tile.drones)
                if total > 0:
                    angle_step = 360 / total if total > 1 else 0
                    radius = cell // 6
                    for d_idx, drone in enumerate(tile.drones):
                        # radial offset for multiple drones on same tile
                        offset = pygame.math.Vector2(0, 0)
                        if total > 1:
                            offset = pygame.math.Vector2(1, 0).rotate(angle_step * d_idx) * (cell // 4)
                        center = (rect.centerx + int(offset.x), rect.centery + int(offset.y))

                        circle_color = getattr(drone, "render_color", gui["drone_color"])
                        pygame.draw.circle(self.screen, circle_color, center, radius)

                        # opaque ID text (prevents path bleed-through)
                        label = str(drone.id)
                        shadow = self._font.render(label, True, (0, 0, 0), circle_color)
                        text_surf = self._font.render(label, True, gui["text_color"], circle_color)
                        shadow_rect = shadow.get_rect(center=(center[0] + 1, center[1] + 1))
                        text_rect = text_surf.get_rect(center=center)
                        self.screen.blit(shadow, shadow_rect)
                        self.screen.blit(text_surf, text_rect)

        # 4) highlight current drone while thinking
        if getattr(self.sim, "_thinking", False):
            current_drone = self.sim.drones[self.sim.turn - 1] # 0-based turn index
            if current_drone:
                cx, cy = current_drone.position
                y_flip = gh - 1 - cy
                hi_rect = pygame.Rect(cx * (cell + m) + m, y_flip * (cell + m) + m, cell, cell)
                pygame.draw.rect(self.screen, (255, 215, 0), hi_rect, 2)

        # 5) sidebar and flip
        self._draw_sidebar()
        pygame.display.flip()


In [13]:
# Simulation
# =========================
class Simulation:
    def __init__(self, game_index: int = 1, total_games: int = 1):
        if CONFIG["simulation"].get("headless", False):
            os.environ.setdefault("SDL_VIDEODRIVER", "dummy")

        self.turn = 1
        self.round = 1
        rules_path = CONFIG.get("rules_path", CONFIG.get("rules_path", "rules.txt"))
        with open(rules_path, "r", encoding="utf-8") as f:
            self.rules = f.read().replace("NUMBER_OF_DRONES", str(CONFIG["simulation"]["num_drones"]))

        self.grid_size = (CONFIG["board"]["width"], CONFIG["board"]["height"])
        self.max_rounds = CONFIG["simulation"]["max_rounds"]
        self.num_drones = CONFIG["simulation"]["num_drones"]
        self.models = CONFIG["simulation"]["models"]
        self.model_index = CONFIG["simulation"]["model_index"]
        self.model = self.models[self.model_index]
        LOGGER.log(f"Using model: {self.model}")

        if not isinstance(total_games, int) or total_games <= 0:
            total_games = 1
        if not isinstance(game_index, int) or game_index <= 0:
            game_index = 1
        self.total_games = total_games
        self.game_index = min(game_index, self.total_games)
        self.gui = None
        if CONFIG["simulation"].get("use_gui", True):
            self.gui = _SimulationGUI(self)


        self.planning_rounds = int(CONFIG["simulation"].get("planning_rounds", 2))
        self.plans: Dict[int, List[str]] = {}
        self._last_logged_plans: Dict[int, str] = {}

        self.board = [[_Tile(x, y) for y in range(self.grid_size[1])] for x in range(self.grid_size[0])]
        self.figures: List[_Figure] = []
        self.drones: List[_Drone] = []

        self.gt_edges = []
        self.reported_edges = []
        self.correct_edge_counter = 0
        self.false_edge_counter = 0
        self.score = 0.0
        self.broadcast_count = 0

        self._create_figures()
        if self.figures:
            white_king = next((figure for figure in self.figures if figure.color == "white" and figure.figure_type == "king"), self.figures[0])
            self.drone_base = white_king.position
        else:
            self.drone_base = (0, 0)

        self._create_drones()

        self.executor = ThreadPoolExecutor(max_workers=1)
        self._current_future: Optional[Future] = None
        self._thinking = False
        self._abort_requested = False

    def _progress_string(self, round_num: Optional[int] = None, turn_num: Optional[int] = None) -> str:
        round_value = round_num if round_num is not None else self.round
        turn_value = turn_num if turn_num is not None else self.turn
        round_value = round_value if round_value else 1
        turn_value = turn_value if turn_value else 1
        return f"{self.game_index}.{round_value}.{turn_value}/{self.total_games}.{self.max_rounds}.{self.num_drones}"

    def progress_caption(self, round_num: Optional[int] = None, turn_num: Optional[int] = None) -> str:
        return f"Simulation - Round {self._progress_string(round_num, turn_num)}"

    # Figures
    def _create_figures(self):
        LOGGER.log("Creating figures based on configuration.")
        self.figures = []
        figures_cfg = CONFIG.get("figures", {})
        sim_cfg = CONFIG.get("simulation", {})
        W, H = self.grid_size
        randomize = bool(sim_cfg.get("randomize_figures", False) or sim_cfg.get("random_seed", None) is not None)
        rng = random.Random(sim_cfg.get("random_seed", None))
        if randomize:
            try:
                all_tiles = [(x,y) for x in range(W) for y in range(H)]
                rng.shuffle(all_tiles)
                cursor = 0
                req = []
                for color in COLORS:
                    for ftype in FIGURE_TYPES:
                        lst = figures_cfg.get(color, {}).get(ftype, [])
                        cnt = len(lst) if lst else (1 if ftype in ("king", "queen") else (2 if ftype in ("rook","bishop","knight") else 3))
                        req.append((color, ftype, cnt))
                out = {c: {t: [] for t in FIGURE_TYPES} for c in COLORS}
                for color, ftype, cnt in req:
                    picks = all_tiles[cursor:cursor+cnt]; cursor += cnt
                    out[color][ftype] = [list(p) for p in picks]
                figures_cfg = out
                LOGGER.log(f"Figure positions RANDOMIZED (seed={sim_cfg.get('random_seed', None)}).")
            except Exception as e:
                LOGGER.log(f"Randomization failed ({e}); using configured positions.")
        for color in COLORS:
            for figure_type in FIGURE_TYPES:
                for position in figures_cfg.get(color, {}).get(figure_type, []):
                    self.figures.append(_Figure(tuple(position), color, figure_type))
        for f in self.figures:
            self.board[f.position[0]][f.position[1]].set_figure(f)

        # Calculate targeting map
        for x in range(self.grid_size[0]):
            for y in range(self.grid_size[1]):
                self.board[x][y].reset_targeted_by_amounts()
        for figure in self.figures:
            figure.calculate_figure_targets(self.board)
        for figure in self.figures:
            for (tx, ty) in figure.target_positions:
                self.board[tx][ty].add_targeted_by_amount(figure.color, 1)

        # Compute defended_by / attacked_by counts
        for figure in self.figures:
            tile = self.board[figure.position[0]][figure.position[1]]
            if figure.color == "white":
                figure.defended_by = tile.targeted_by["white"]; figure.attacked_by = tile.targeted_by["black"]
            else:
                figure.defended_by = tile.targeted_by["black"]; figure.attacked_by = tile.targeted_by["white"]

        # Compute GT Edges
        self.gt_edges = []
        for figure in self.figures:
            for (x_tgt, y_tgt) in figure.target_positions:
                target_figure = self.board[x_tgt][y_tgt].figure
                if target_figure is not None:
                    edge = format_edge(figure.figure_type, figure.color, target_figure.color, [figure.position, target_figure.position])
                    self.gt_edges.append(edge)
        LOGGER.log(f"GT Edges computed: {len(self.gt_edges)}")

    # Drones
    def _create_drones(self):
        LOGGER.log(f"Creating {self.num_drones} drones.")
        for drone_index in range(self.num_drones):
            new_drone = _Drone(id=drone_index+1, position=self.drone_base, model=self.model, rules=self.rules, sim=self)
            # unique color by rotating hue
            hue_deg = (drone_index / max(1, self.num_drones)) * 360.0
            new_drone.render_color = hsv_to_rgb255(hue_deg, 0.85, 0.95)  # vivid, bright
            self.drones.append(new_drone)
        base_tile = self.board[self.drone_base[0]][self.drone_base[1]]
        for new_drone in self.drones:
            base_tile.add_drone(new_drone)

    # Edges/score
    def report_edges(self, edges: List[str]):
        for edge in edges:
            if edge not in self.reported_edges:
                self.reported_edges.append(edge)
        correct_edge_counter = 0; false_edge_counter = 0
        for edge in self.reported_edges:
            if edge in self.gt_edges:
                correct_edge_counter += 1
            else:
                false_edge_counter += 1
        self.correct_edge_counter = correct_edge_counter
        self.false_edge_counter = false_edge_counter
        self.score = correct_edge_counter - false_edge_counter

    def _log_final_summary(self):
        LOGGER.log("#"*60)
        LOGGER.log("FINAL EDGE SUMMARY")
        LOGGER.log(f"Reported edges: {len(self.reported_edges)}")
        LOGGER.log(f"  - Correct:    {self.correct_edge_counter}")
        LOGGER.log(f"  - False:      {self.false_edge_counter}")
        LOGGER.log(f"Score:          {self.score} / {len(self.gt_edges)} = {self.score / max(1, len(self.gt_edges)):.2%}")

        LOGGER.log("\nEdge summary:")
        correct_edges = []; false_edges = []
        for edge in self.reported_edges:
            if edge in self.gt_edges:
                correct_edges.append(edge)
            else:
                false_edges.append(edge)
        for edge in correct_edges:
            LOGGER.log(f"  CORRECT: {edge}")
        for edge in false_edges:
            LOGGER.log(f"  FALSE:   {edge}")

        if self.gui:
            try:
                self.gui.draw_field()
            except Exception:
                pass
        self.post_info("=== FINAL EDGE SUMMARY ===")
        self.post_info(f"Reported: {len(self.reported_edges)}  Correct: {self.correct_edge_counter}  False: {self.false_edge_counter}  Score: {self.score} / {len(self.gt_edges)}")

    # GUI/log helper
    def post_info(self, msg: str):
        if hasattr(self, "gui") and self.gui:
            self.gui.post_info(msg)

    def _log_round_mission_plans(self, round_number: int) -> None:
        try:
            updates = []
            for drone in self.drones:
                plan = getattr(drone, "mission_plan", [])
                snapshot = {
                    "sector": getattr(drone, "assigned_sector", None),
                    "plan": plan,
                }
                try:
                    signature = json.dumps(snapshot, sort_keys=True, default=lambda o: getattr(o, "__dict__", str(o)))
                except Exception:
                    signature = json.dumps(str(snapshot))
                previous = self._last_logged_plans.get(drone.id)
                if signature != previous:
                    updates.append((drone, plan, signature))
            if not updates:
                return

            LOGGER.log(f"Mission plan update for round {round_number}:")
            for drone, plan, signature in updates:
                self._last_logged_plans[drone.id] = signature
                LOGGER.log(f"  Drone {drone.id} mission plan:")
                try:
                    LOGGER.log(f"    Sector: {drone.sector_summary()}")
                except Exception:
                    pass
                for entry in plan:
                    if isinstance(entry, dict) and entry.get("leg_start") and entry.get("leg_end"):
                        start_vec = entry.get("leg_start")
                        end_vec = entry.get("leg_end")
                        start = cartesian_to_chess(tuple(start_vec)) if start_vec else "?"
                        end = cartesian_to_chess(tuple(end_vec)) if end_vec else "?"
                        LOGGER.log(
                            f"    Leg {entry.get('leg_id','?')}: {start} -> {end} "
                            f"(len={entry.get('length','?')}, orient={entry.get('orientation','?')})"
                        )
                for line in pprint.pformat(plan, indent=2, width=120).splitlines():
                    LOGGER.log(f"    {line}")
        except Exception as exc:
            LOGGER.log(f"Error logging mission plans: {exc}")

    # Turn orchestration
    def _try_finish_drone_turn(self, drone: _Drone) -> bool:
        if self._current_future is None or not self._current_future.done():
            return False

        try:
            messages = self._current_future.result()
            result = json.loads(messages[-1]["content"])
            LOGGER.log(f"Drone {drone.id} response:\n{pprint.pformat(result, indent=4, width=200)}")

            self.post_info(f"Drone {drone.id}:")
            self.post_info(f"Rationale: {result.get('rationale','')}")
            action = result.get("action", "wait")

            if action == "move":
                direction = (result.get("direction") or "").lower()
                legal_directions = [lms["direction"] for lms in drone._legal_movement_steps()]
                if direction not in legal_directions:
                    self.post_info(f"Invalid direction: '{direction}', allowed={legal_directions} -> Aborted movement")
                elif drone._move(direction):
                    self.post_info(f"Move {direction} to {cartesian_to_chess(drone.position)}")
            elif action == "broadcast":
                raw_message = result.get("message")
                msg = ""
                plan_payload = None
                if isinstance(raw_message, str):
                    msg = raw_message.strip()
                    if msg:
                        try:
                            plan_payload = json.loads(msg)
                        except Exception:
                            plan_payload = None
                elif isinstance(raw_message, (dict, list)):
                    plan_payload = raw_message
                    try:
                        msg = json.dumps(raw_message)
                    except Exception:
                        msg = str(raw_message)
                else:
                    msg = ""

                if not msg:
                    self.post_info("ERROR: Empty broadcast.")
                else:
                    self.post_info("Broadcast")
                try:
                    self.broadcast_count += 1
                except Exception:
                    pass
                self.post_info(msg or "<no message>")
                if plan_payload:
                    drone._apply_plan_directive(plan_payload)

                tile = self.board[drone.position[0]][drone.position[1]]
                for target_drone in tile.drones:
                    if target_drone.id != drone.id:
                        target_drone.rx_buffer += f"Drone {drone.id} broadcasted: {msg}\n"
                        if plan_payload:
                            target_drone._apply_plan_directive(plan_payload)
                        drone._provide_intelligence_to(target_drone)
            else:
                self.post_info("Wait")

            # persist memory + mark visited
            mem_txt = (result.get("memory") or "").strip()
            if mem_txt: drone.memory = mem_txt
            vx, vy = drone.position
            token = f"VISITED:{vx},{vy}"
            if token not in drone.memory:
                drone.memory += ("" if drone.memory.endswith("\n") else "\n") + token

            # edge logging
            self.post_info("\n")
        except Exception as e:
            LOGGER.log(f"Error finishing Drone {drone.id}'s turn: {e}")

        self._thinking = False
        self._current_future = None
        return True

    # Main loop
    def run_simulation(self):
        max_rounds = CONFIG["simulation"].get("max_rounds", 10)
        use_gui = CONFIG["simulation"].get("use_gui", True) and hasattr(self, "gui")

        running = True
        clock = None
        if use_gui:
            clock = pygame.time.Clock()
            pygame.display.set_caption(self.progress_caption(round_num=1, turn_num=1))
            self.gui.draw_field()

        current_round = 1
        drone_index = 0
        pending = False

        try:
            while running:
                if use_gui:
                    for event in pygame.event.get():
                        if event.type == pygame.QUIT:
                            running = False
                            self._abort_requested = True
                        elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                            running = False
                            self._abort_requested = True

                current_drone = self.drones[drone_index]
                if not pending:
                    if current_round > max_rounds: break
                    self.round = current_round
                    self.turn = drone_index + 1
                    caption = self.progress_caption(current_round, self.turn)
                    LOGGER.log('#'*50); LOGGER.log(caption)
                    if drone_index == 0:
                        self._log_round_mission_plans(current_round)
                    if use_gui:
                        pygame.display.set_caption(caption)
                    self._thinking = True
                    self._current_future = self.executor.submit(current_drone.generate_full_model_response)
                    pending = True

                if pending:
                    if self._try_finish_drone_turn(current_drone):
                        drone_index += 1
                        if drone_index >= self.num_drones:
                            drone_index = 0
                            current_round += 1
                        pending = False

                if use_gui:
                    self.gui.draw_field()
                    clock.tick(60)
        except KeyboardInterrupt:
            LOGGER.log("KeyboardInterrupt received â€” shutting down gracefully.")
            running = False
        finally:
            try:
                self._log_final_summary()
            except Exception:
                pass
            self.shutdown()

    def shutdown(self):
        try:
            if getattr(self, "_current_future", None) and not self._current_future.done():
                self._current_future.cancel()
        except Exception:
            pass
        try:
            if getattr(self, "executor", None):
                self.executor.shutdown(wait=False, cancel_futures=True)
                self.executor = None
        except Exception:
            pass
        try:
            if self.gui:
                # draw one last frame to ensure latest state is shown
                self.gui.draw_field()
                self.gui.save_screenshot()  # default path in ./screenshots
        except Exception as ex:
            LOGGER.log(f"Error during final GUI draw: {ex}")
        pygame.display.quit()
        pygame.quit()
        LOGGER.log("Clean shutdown complete.")


In [14]:
# Main
# =========================
RUN_EXPORTS = []

if __name__ == "__main__":
    seeds = list(seed_list or [None])
    total_games = max(1, len(seeds))
    for game_index, seed in enumerate(seeds, start=1):
        LOGGER.log(f"==== Running seed {seed} (game {game_index}/{total_games}) ====")
        try:
            set_all_seeds(seed)
        except Exception:
            pass
        CONFIG = load_config("config.json")
        if "simulation" in CONFIG:
            CONFIG["simulation"]["random_seed"] = seed
        try:
            LOGGER.log("Launching simulation.")
            run_started = time.time()
            SIM = Simulation(game_index=game_index, total_games=total_games)
            SIM.run_simulation()
            runtime_s = time.time() - run_started
            finished_at = datetime.now().isoformat()
            if getattr(SIM, "_abort_requested", False):
                LOGGER.log("GUI closed by user - stopping remaining seeds.")
                break
            run_entry = {
                "sim": SIM,
                "config": CONFIG,
                "seed": seed,
                "runtime_s": runtime_s,
                "timestamp": finished_at,
            }
            RUN_EXPORTS.append(run_entry)
            try:
                persist_run_results([run_entry])
            except Exception as exc:
                LOGGER.log(f"Failed to update results.csv: {exc}")
        except KeyboardInterrupt:
            LOGGER.log("Interrupted by user (Ctrl+C).")
            try:
                SIM.shutdown()
            except Exception:
                pass
            break
        except Exception as exc:
            LOGGER.log(f"Simulation error: {exc}")
        finally:
            try:
                SIM.shutdown()
            except Exception:
                pass


INFO:root:[2025-11-23 17:08:34] (+0.217s) ==== Running seed 0 (game 1/100) ====
INFO:root:[2025-11-23 17:08:38] (+3.628s) Load Config: config.json
INFO:root:[2025-11-23 17:08:38] (+0.001s) Launching simulation.
INFO:root:[2025-11-23 17:08:38] (+0.001s) Using model: llama3.1:8b


Global seed set to 0


INFO:root:[2025-11-23 17:08:38] (+0.189s) Creating figures based on configuration.
INFO:root:[2025-11-23 17:08:38] (+0.002s) Figure positions RANDOMIZED (seed=0).
INFO:root:[2025-11-23 17:08:38] (+0.001s) GT Edges computed: 59
INFO:root:[2025-11-23 17:08:38] (+0.000s) Creating 2 drones.
INFO:root:[2025-11-23 17:08:38] (+0.001s) - CORRECT: Kd6-d5
INFO:root:[2025-11-23 17:08:38] (+0.000s) - CORRECT: Kd6xc5
INFO:root:[2025-11-23 17:08:38] (+0.004s) ##################################################
INFO:root:[2025-11-23 17:08:38] (+0.001s) Simulation - Round 1.1.1/100.32.2
INFO:root:[2025-11-23 17:08:38] (+0.000s) Mission plan update for round 1:
INFO:root:[2025-11-23 17:08:38] (+0.000s)   Drone 1 mission plan:
INFO:root:[2025-11-23 17:08:38] (+0.001s)     Sector: Full board (a8 -> h1)
INFO:root:[2025-11-23 17:08:38] (+0.000s)     Leg 1: d6 -> a6 (len=3, orient=horizontal)
INFO:root:[2025-11-23 17:08:38] (+0.001s)     Leg 2: a6 -> a8 (len=2, orient=vertical)
INFO:root:[2025-11-23 17:08:38

Context length (chars): 8692


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:root:[2025-11-23 17:08:47] (+8.570s) Drone 1 response:
{   'action': 'broadcast',
    'memory': 'Collected figure information: d6: white king; Assigned coverage sector: Full board (a8 -> h1); Broadcast Rx Buffer: Coverage plan broadcast template; Sector directive: Full board (a8 '
              '-> h1)',
    'message': {   'plan': {   'assignments': [{'drone': 1, 'sector': {'lower_right': 'd1', 'upper_left': 'a8'}}, {'drone': 2, 'sector': {'lower_right': 'h1', 'upper_left': 'e8'}}],
                               'rendezvous': {'tile': 'd6', 'turn': 31}}},
    'rationale': 'As the first drone, I will broadcast a coverage plan to assign sectors to both drones (including myself) and specify a rendezvous tile on the penultimate round.'}
INFO:root:[2025-11-23 17:08:47] (+0.000s) Drone 1 adopted sector Full board (a8 -> d1)
INFO:root:[2025-11-23 17:08:47] (+0.000s) Drone 1 adopted rendezvous d6 by turn 31
I

...

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:root:[2025-11-23 17:12:58] (+1.014s) Drone 2 response:
{   'action': 'move',
    'direction': 'east',
    'memory': 'Collected figure information: d6: white king; Assigned coverage sector: Full board (e8 -> h1); Rendezvous directive: d6 on turn 31; Broadcast Rx Buffer: Drone 1 broadcasted: {',
    'message': None,
    'rationale': 'I will move east to e6, as it is the best choice according to the decision support, allowing me to close distance to leg start, align with coverage leg, entered coverage leg '
                 'corridor, progress along current leg, closer to waypoint, on schedule, and moving toward assigned sector.'}
INFO:root:[2025-11-23 17:12:58] (+0.001s) Drone 2 moved to e6.
INFO:root:[2025-11-23 17:12:58] (+0.016s) ##################################################
INFO:root:[2025-11-23 17:12:58] (+0.000s) Simulation - Round 4.2.1/100.32.2
INFO:root:[2025-11-23 17:12:58] (+0.000s) Missi

Context length: 1936 tokens (8056 chars)
Tokens per second: 205.48 tok/s (139 completion tokens)
Context length (chars): 8082


INFO:root:[2025-11-23 17:12:59] (+0.607s) ############################################################
INFO:root:[2025-11-23 17:12:59] (+0.001s) FINAL EDGE SUMMARY
INFO:root:[2025-11-23 17:12:59] (+0.001s) Reported edges: 5
INFO:root:[2025-11-23 17:12:59] (+0.000s)   - Correct:    5
INFO:root:[2025-11-23 17:12:59] (+0.001s)   - False:      0
INFO:root:[2025-11-23 17:12:59] (+0.000s) Score:          5 / 66 = 7.58%
INFO:root:[2025-11-23 17:12:59] (+0.001s) 
Edge summary:
INFO:root:[2025-11-23 17:12:59] (+0.000s)   CORRECT: Kd6-c6
INFO:root:[2025-11-23 17:12:59] (+0.000s)   CORRECT: Kd6-d5
INFO:root:[2025-11-23 17:12:59] (+0.001s)   CORRECT: Kd6-c5
INFO:root:[2025-11-23 17:12:59] (+0.000s)   CORRECT: Kd6xe5
INFO:root:[2025-11-23 17:12:59] (+0.000s)   CORRECT: Kd6xc7
INFO:root:[2025-11-23 17:12:59] (+0.025s) Saved GUI screenshot: screenshots\last_run.png
INFO:root:[2025-11-23 17:12:59] (+0.064s) Clean shutdown complete.
INFO:root:[2025-11-23 17:12:59] (+0.001s) GUI closed by user - stoppin

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Context length: 1903 tokens (8082 chars)
Tokens per second: 210.32 tok/s (123 completion tokens)
